In [1]:
# Run Baseline and Outsourcers all together

folder = r"C:\Users\sps55207\Desktop\Outsourcing\Figures"
file_ = r"\Output.txt"



# Delete current contents of folder
import os
import glob

files = glob.glob(folder + r"\*")
for f in files:
    os.remove(f)
    
f = open(folder + file_, "w")

# Baseline Decentralized

# Create a baseline version of the model following L+S pg 953

import numpy as np
import matplotlib.pyplot as plt
import quantecon as qe
from numba import njit, prange, vectorize
from scipy import optimize
from IPython.core.debugger import set_trace

# Want to vectorize cost of vacancy, marginal cost of vacancy, and inverse marginal cost of vacancy
@vectorize
def C(v, gamma, k_grid):
    return k_grid * v**gamma

@vectorize
def c(v, gamma, k_grid):
    if v > 0:
        return k_grid * gamma * v**(gamma - 1)
    else:
        return 0
     
@vectorize
def c_inv(c, gamma, k_grid):
    if c > 0:
        return (c / gamma / k_grid)**(1 / (gamma - 1))
    else:
        return 0
    
# Set up the baseline environment, starting with a discrete model
# For now, have entry costs be exponential, matching probability be q=phi * theta**(1 / 2) 
class BaselineDiscrete:
    """
    B is discount rate beta
    r is interest rate defined by beta
    b is home production
    delta is job loss rate
    eta is worker bargaining power    
    phi is effectiveness of matching function
    q is firm's matching function, worker's is p=theta*q(theta)
    y_min, y_max, y_grid_size determine productivity grid
    gamma is steepness of marginal cost, k_grid is distribution of firm entry costs
    q is matching function of firms
    p is matching function of workers    
    """
    def __init__(self, B, b, delta, eta, phi, y_min, y_max, y_grid_size, gamma, k_grid):
        
        self.B, self.b, self.delta, self.eta, self.gamma, self.k_grid = B, b, delta, eta, gamma, k_grid
        self.r = 1 / B - 1
        
        self.y_grid = np.linspace(y_min, y_max, y_grid_size)
        self.y_grid_size = y_grid_size        
        
        self.q = njit(lambda x: min(phi * x**(-1 / 2), 1))
        self.p = njit(lambda x: min(phi * x**(1 / 2), 1))
        self.q_p = njit(lambda x: -phi / 2 * x**(-3 / 2))
        
        
# Set up functions of problem
def b_operator_factory(bl):
    "Use a baseline case bl and create a function to find next vacancy and unemployment iterations"
    
    B, r, b, delta, eta, gamma, k_grid = bl.B, bl.r, bl.b, bl.delta, bl.eta, bl.gamma, bl.k_grid
    y_grid, y_grid_size = bl.y_grid, bl.y_grid_size
    q, p = bl.q, bl.p
    
    # Find wages for each postion and market tightness theta
    @njit()
    def wages(v_grid, u):
        w_grid = b + eta * (y_grid - b + np.sum(c(v_grid, gamma, k_grid) * v_grid) / u)
        theta = np.sum(v_grid) / u
        return w_grid, theta
    
    # Find next v_grid and u
    @njit()
    def iterate(v_grid, u):  

        w_grid, theta = wages(v_grid, u) 

        # Use w_grid to find out how many of firm i wants to enter (cannot be below 0)
        v_grid_new = c_inv(q(theta) * (y_grid - w_grid) / (r + delta), gamma, k_grid)
        
#         # Update u_new using steady state rule
#         u_new = delta / (delta + p(theta))

        # Try updating u_new using unemployed workers
        n_grid_new = q(theta) * v_grid_new / delta
        u_new = max(1 - np.sum(n_grid_new), 1e-5)
        
        return v_grid_new, u_new 
    
    return iterate, wages

def solve_model(bl, tol=1e-4, max_iter=1000, verbose=True, print_skip=25, slow=1/100):
    
    iterate, wages = b_operator_factory(bl)
    
    # Set up initial guesses and loop parameters
    v_grid = np.ones(bl.y_grid_size) / bl.y_grid_size
    u = 0.05
    i = 0
    err = tol + 1    
    
    # Update v_grid and u each period
    while i < max_iter and err > tol:
        v_grid_new, u_new = iterate(v_grid, u)
        err_v = np.max(np.abs(v_grid_new - v_grid))
        err_u = np.abs(u_new - u)
        err = max(err_v, err_u)
        i += 1
        
        if verbose and i % print_skip == 0:
            print(f'Error at iter {i} is {err}')
        
        v_grid = slow * v_grid_new + (1 - slow) * v_grid 
        u = slow * u_new + (1 - slow) * u
        
    if i == max_iter:
        print("Failed to Converge")
        
    if verbose and i < max_iter:
        print(f'Converged in {i} iterations')
    
    return v_grid_new, u_new

# Can check code against theoretical results by looking at spread and number of entrants
# For spread of entrants, compare each firms entry to highest. Graph results
# Keep in mind that spread_theory doesn't know that vacancies cannot be negative
def b_spread(bl, v_grid, u,
           save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Baseline\Spread'):
    
    gamma, k_grid, eta, r, delta = bl.gamma, bl.k_grid, bl.eta, bl.r, bl.delta
    q, y_grid = bl.q, bl.y_grid
    
    theta = np.sum(v_grid) / u
    
    # Find highest positive entry
    i_m = np.argmax(y_grid[v_grid > 0])    
    
    spread_prod = (c(v_grid[i_m], gamma, k_grid[i_m]) - c(v_grid, gamma, k_grid)) * (r + delta) / ((1 - eta) * q(theta))
    spread_theory = (y_grid[i_m] - y_grid) 
    spread_theory[v_grid == 0] = spread_prod[v_grid == 0]
    dif = spread_theory - spread_prod
    
    fig, ax = plt.subplots()
    ax.plot(y_grid, spread_prod, lw=2, alpha=0.7, label='Production')
    ax.plot(y_grid, spread_theory, lw=2, alpha=0.7, label='Theoretical')
    ax.set(xlabel='$y$', ylabel='Spread', title='Spread of Vacancies')
    ax.legend(loc='best')
    
    
    if save:
        fig.savefig(folder + file + '.pdf')        
        plt.close()
    else:
        plt.show()   
        
    fig, ax = plt.subplots()
    ax.plot(y_grid, dif, lw=2, alpha=0.7)
    ax.set(xlabel='$y$', ylabel='Spread', title='Difference in Spread of Vacancies')
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    
    if save:
        fig.savefig(folder + file + '_dif.pdf')
        plt.close()
    else:
        plt.show()  

# For number of vacancies, just have two numbers to compare
def b_entry(bl, v_grid, u):
    
    b, gamma, k_grid, eta, r, delta = bl.b, bl.gamma, bl.k_grid, bl.eta, bl.r, bl.delta
    q, p, y_grid = bl.q, bl.p, bl.y_grid
    
    theta = np.sum(v_grid) / u
    
    entry_theory = np.sum(v_grid * (y_grid - b))
    entry_code = ((r + delta + eta * theta * q(theta)) * np.sum(v_grid * c(v_grid, gamma, k_grid))
                  / ((1 - eta) * q(theta)))
    
    f.write(f'Entry from code gives {entry_code:.4f}; Entry from theory gives {entry_theory:.4f} \n')
    
    # While doing this, also check if u is consistent with LOM
    u_t = delta / (delta + p(theta))    
    f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
    # Probably related to whether p and q are < 1. Check these
    f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
# Create a pdf of wage distribution
def b_wage_dist(v_grid, w_grid,
           save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Baseline\Wage_Dist'):
    
    w_plot = w_grid[v_grid > 0]
    v_pdf = v_grid[v_grid > 0] / sum(v_grid)
    
    fig, ax = plt.subplots()
    ax.plot(w_plot, v_pdf, alpha=0.7, color='b')
    ax.fill_between(w_plot, 0, v_pdf, alpha=0.1, color='b')
    ax.set(xlabel='Wages', ylabel='Density')
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    if save:
        fig.savefig(folder + file + '.pdf')
    else:
        plt.show() 
    
############################################################################################################


# Now solve the baseline Planner's Problem. Takes the same baseline discrete class.
# Try to calculate shadow cost of a vacancy for every type and use this to determine optimal amount

def p_operator_factor(bl):
    "Use a baseline case bl and create a function to update Lagrange multipliers and solve planner's problem"
    
    B, r, b, delta, eta, gamma, k_grid = bl.B, bl.r, bl.b, bl.delta, bl.eta, bl.gamma, bl.k_grid
    y_grid, y_grid_size = bl.y_grid, bl.y_grid_size
    q, q_p, p = bl.q, bl.q_p, bl.p
        
    # update v_grid and n_grid
    @njit()
    def update_grids(v_grid, u, lam, slow=1):
                
        theta = np.sum(v_grid) / u
        
        # lam is the Lagrangian on each firm
        lam_new = (y_grid - b + (theta * q_p(theta) / u) * np.sum(v_grid * lam)) / (r + delta)
        lam_new = lam_new * slow + lam * (1 - slow)
        
        # Given a lam_new, repeatedly update v_grid and n_grid hopefully will be faster
        for j in range(2e2):      
            theta = np.sum(v_grid) / u
            v_grid_new = c_inv(lam_new * q(theta) + ((q_p(theta) / u) * np.sum(v_grid * lam_new)), gamma, k_grid)
            
            n_grid_new = q(theta) * v_grid_new / delta
            u_new = max(1 - np.sum(n_grid_new), 1e-5)
            
            v_grid = v_grid_new * slow + v_grid * (1 - slow)
            u = u_new * slow + u * (1 - slow)      
        
        return v_grid_new, u_new, lam_new
    
    return update_grids
        
# Solve model
def solve_p_model(bl, guess, tol=1e-4, max_iter=10000, slow=1, verbose=True, print_skip=25):   

    update_grids = p_operator_factor(bl)

    r, b, delta, y_grid = bl.r, bl.b, bl.delta, bl.y_grid
    # Set up initial guesses and loop parameters
    v_grid, u = guess
    theta = np.sum(v_grid) / u
    
    # Carry around lam to make re-estimating faster
    lam = (y_grid - b) / (r + delta)
    
    i = 0
    err = tol + 1 
    
    # Update v_grid, n_grid, and lamda each period
    while i < max_iter and err > tol:
        v_grid_new, u_new, lam_new = update_grids(v_grid, u, lam, slow)
        err_v = np.max(np.abs(v_grid_new - v_grid))
        err_u = np.abs(u_new - u)
        err = max(err_v, err_u)
        i += 1
        
        if verbose and i % print_skip == 0:
            print(f'Error at iter {i} is {err}')
            print(v_grid[-1])
        
        v_grid = v_grid_new            
        u = u_new
        lam = lam_new        
        
    if i == max_iter:
        print("Failed to Converge")
        
    if verbose and i < max_iter:
        print(f'Converged in {i} iterations')
    
    return v_grid_new, u_new
    
# Can check code against theoretical results by looking at spread and number of entrants
# For spread of entrants, compare each firms entry to highest. Graph results
# Keep in mind that spread_theory doesn't know that vacancies cannot be negative
def p_spread(bl, v_grid, u,
           save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Baseline\Planner_Spread'):
    
    gamma, k_grid, eta, r, delta = bl.gamma, bl.k_grid, bl.eta, bl.r, bl.delta
    q, y_grid = bl.q, bl.y_grid
        
    theta = np.sum(v_grid) / u

    # Find highest positive entry (if it exists)
    if any(v_grid > 0):
        i_m = np.argmax(y_grid[v_grid > 0]) 
    else:
        i_m = -1 
    
    spread_prod = (c(v_grid[-1], gamma, k_grid[-1]) - c(v_grid, gamma, k_grid)) * (r + delta) / q(theta)
    spread_theory = (y_grid[-1] - y_grid) 
    spread_theory[v_grid == 0] = spread_prod[v_grid == 0]
    dif = spread_theory - spread_prod
    
    fig, ax = plt.subplots()
    ax.plot(y_grid, spread_prod, lw=2, alpha=0.7, label='Production')
    ax.plot(y_grid, spread_theory, lw=2, alpha=0.7, label='Theoretical')
    ax.set(xlabel='$y$', ylabel='Spread', title='Spread of Vacancies')
    ax.legend(loc='best')
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    if save:
        fig.savefig(folder + file + '.pdf')
        plt.close()
    else:
        plt.show()   
        
    fig, ax = plt.subplots()
    ax.plot(y_grid, dif, lw=2, alpha=0.7)
    ax.set(xlabel='$y$', ylabel='Spread', title='Difference in Spread of Vacancies')
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    
    if save:
        fig.savefig(folder + file + '_dif.pdf')
        plt.close()
    else:
        plt.show()  

# For number of vacancies, just have two numbers to compare
def p_entry(bl, v_grid, u):
    
    b, gamma, k_grid, eta, r, delta = bl.b, bl.gamma, bl.k_grid, bl.eta, bl.r, bl.delta
    q, q_p, p, y_grid = bl.q, bl.q_p, bl.p, bl.y_grid
    
    theta = np.sum(v_grid) / u
    alpha = -(theta * q_p(theta)) / q(theta)
    
    entry_theory = np.sum(v_grid * (y_grid - b))
    entry_prod = ((r + delta + alpha * theta * q(theta)) * np.sum(v_grid * c(v_grid, gamma, k_grid)) 
                  / ((1 - alpha) * q(theta)))
    
    f.write(f'Entry from production gives {entry_prod:.4f}; Entry from theory gives {entry_theory:.4f} \n')
    
    # While doing this, also check if u is consistent with LOM
    u_t = delta / (delta + bl.p(theta))    
    f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
    # Probably related to whether p and q are < 1. Check these
    f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
        
###########################################

# Do the same for outsourcers
@vectorize
def C_h(v, gamma_h, k_h):
    return k_h * v**gamma_h

@vectorize
def c_h(v, gamma_h, k_h):
    if v > 0:
        return k_h * gamma_h * v**(gamma_h - 1)
    else:
        return 0
     
@vectorize
def c_h_inv(c, gamma_h, k_h):
    if c > 0:
        return (c / gamma_h / k_h)**(1 / (gamma_h - 1))
    else:
        return 0
    
# Set up the baseline environment, starting with a discrete model
class OutsourcerDiscrete:
    """
    B is discount rate beta
    r is interest rate defined by beta
    b is home production
    delta is job loss rate
    delta_hat is firm destruction rate <= delta
    eta is workers bargaining power with firm
    eta_h is workers bargaining power with outsourcer
    phi is effectiveness of matching function
    q is firm's matching function, worker's is p=theta*q(theta)
    y_min, y_max, y_grid_size determine productivity grid
    gamma is steepness of marginal cost, k_grid is level of entry cost
    gamma_h is steepness of marginal cost for outsourcer, k_h is level of cost
    q is matching function of firms
    p is matching function of workers
    
    q_p is derivative of firm's matching function for Planner's
    p_p is derivative of worker's matching function for Planner's
    
    barg is 1 if firms can use outsourcer as outside option with workers, 0 else
    """
    def __init__(
        self, B, b, delta, delta_hat, eta, eta_h, phi,
        y_min, y_max, y_grid_size, gamma, gamma_h, k_grid, k_h, barg=0
    ):
        
        self.B, self.b, self.delta, self.eta, self.gamma, self.k_grid = B, b, delta, eta, gamma, k_grid
        self.delta_hat, self.eta_h, self.gamma_h, self.k_h = delta_hat, eta_h, gamma_h, k_h 
        self.r = 1 / B - 1
        
        self.y_grid = np.linspace(y_min, y_max, y_grid_size)
        self.y_grid_size = y_grid_size        
        
        self.q = njit(lambda x: min(phi * x**(-1 / 2), 1))
        self.p = njit(lambda x: min(phi * x**(1 / 2), 1))
        # Remember, q_p and p_p = 0 if q or p = 1
        self.q_p = njit(lambda x: -phi / 2 * x**(-3 / 2) * (phi * x**(-1 / 2) < 1)) 
        self.p_p = njit(lambda x: phi / 2 * x**(1 / 2) * (phi * x**(1 / 2) < 1))
        
        self.barg = barg
        
# Set up needed functions
def o_operator_factory(out):
    
    r, b, delta, eta, gamma, k_grid = out.r, out.b, out.delta, out.eta, out.gamma, out.k_grid
    delta_hat, eta_h, gamma_h, k_h = out.delta_hat, out.eta_h, out.gamma_h, out.k_h
    y_grid, y_grid_size = out.y_grid, out.y_grid_size
    y_grid_max = max(y_grid)
    q, p = out.q, out.p
    barg = out.barg
    
    # Find wages for each postion
    @njit()
    def wages(v_grid, v_h, u, y_hat, p_h):
        firm_sum = np.sum(c(v_grid[y_grid < y_hat], gamma, k_grid[y_grid < y_hat]) * v_grid[y_grid < y_hat])
        out_sum = c_h(v_h, gamma_h, k_h) * v_h        
        w_grid = (b + eta * (y_grid - b + (firm_sum + (1 - eta) * eta_h / ((1 - eta_h) * eta) * out_sum) / u) 
                  - barg * eta * (r + delta) / (r + delta_hat) * np.maximum(y_grid - p_h, 0))
        omega = b + eta_h * (p_h - b + ((1 - eta_h) * eta / ((1 - eta) * eta_h) * firm_sum + out_sum) / u)
        return w_grid, omega 
     
    # Find y_hat_new using prices
    @njit()
    def get_y_hat(p_h, w_grid, theta):
        
        J = (y_grid - w_grid) / (r + delta)
        J_H = (y_grid - p_h) / (r + delta_hat)
        choice = 1 * (J_H >= q(theta) * J) * (J_H > 0) 
        ind = np.searchsorted(choice, 0, side='right')
        # If y_hat > y_max, return y_max + 1
        if ind < y_grid_size:
            y_hat = y_grid[ind]
        else:
            y_hat = y_grid_max + 1
        return y_hat
    
    # Find quantity supplied and quantity demanded for outsourcing
    @njit()
    def find_QS_QD(v_grid, v_h, u, y_hat):
        
        theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
        # Supply of outsourcing are matched outsourcers, not outsourcer vacancies
        QS = v_h * q(theta) / delta
        # Demand for outsourcing are matched firms, not firm vacancies
        QD = np.sum(v_grid[y_grid >= y_hat]) / delta_hat
        return QS, QD
    
    @njit()
    def iterate(v_grid, v_h, u, y_hat, p_h):
        
        # Set theta and wages
        theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
        w_grid, omega = wages(v_grid, v_h, u, y_hat, p_h)    
        
        v_grid_new = np.empty(y_grid_size)       
        
        # Update y_hat
        y_hat_new = get_y_hat(p_h, w_grid, theta)
        
        # Should the relevant comparison be y_hat or y_hat_new? Make switching easier
        y_comp = y_hat
        
        # Firms who hire own workers (below y_comp)
        v_grid_new[y_grid < y_comp] = (
            c_inv(q(theta) * (y_grid[y_grid < y_comp] - w_grid[y_grid < y_comp]) / (r + delta),
                  gamma, k_grid[y_grid < y_comp]))
        
        # Firms who outsource workers (above y_comp)
        v_grid_new[y_grid >= y_comp] = (
            c_inv((y_grid[y_grid >= y_comp] - p_h) / (r + delta_hat), gamma, k_grid[y_grid >= y_comp]))
        
        # v_h_new chosen using free entry
        v_h_new = c_h_inv(q(theta) * (p_h - omega) / (r + delta), gamma_h, k_h)
        
        # Update u_new using employed workers
        n_grid_b = q(theta) * v_grid_new[y_grid < y_comp] / delta
        u_new = max(1 - np.sum(n_grid_b) - (q(theta) * v_h_new / delta), 1e-6)

        return v_grid_new, v_h_new, u_new, y_hat_new
    
    return iterate, wages, get_y_hat, find_QS_QD

# Solve model given p_h 
def solve_given_p(out, p_h, v_grid, v_h, u, y_hat, tol, max_iter, slow):
    
    iterate, _, _, _ = o_operator_factory(out)
    
    # Set up initial loop parameters
    i = 0
    err = tol + 1    
    
    # Update v_grid, v_h, u, and y_hat each period
    while i < max_iter and err > tol:
        v_grid_new, v_h_new, u_new, y_hat_new = iterate(v_grid, v_h, u, y_hat, p_h)
        err_v = np.max(np.abs(v_grid_new - v_grid))
        err_v_h = np.abs(v_h_new - v_h)
        err_u = np.abs(u_new - u)
        err_y_hat = np.abs(y_hat_new - y_hat)
        err = max(err_v, err_v_h, err_u, err_y_hat)
        i += 1
        
        v_grid = slow * v_grid_new + (1 - slow) * v_grid
        v_h = slow * v_h_new + (1 - slow) * v_h        
        u = slow * u_new + (1 - slow) * u
        y_hat = slow * y_hat_new + (1 - slow) * y_hat
        
    return v_grid_new, v_h_new, u_new, y_hat_new

# Find p_h by bisection
def find_p(
    out, tol, max_iter, p_low, p_high, tol_p, max_iter_p, slow_p=1, verbose=True, print_skip=100
):
    
    iterate, wages, get_y_hat, find_QS_QD = o_operator_factory(out)
    
    b, eta, r, delta, delta_hat = out.b, out.eta, out.r, out.delta, out.delta_hat 
    gamma, gamma_h, k_grid, k_h, q = out.gamma, out.gamma_h, out.k_grid, out.k_h, out.q
    y_grid, y_grid_size = out.y_grid, out.y_grid_size  
    
    # Set up loop parameters    
    err = tol + 1
    j = 0
    test = []
    
    # Find QD - QS for p_low and p_high, make sure they have opposite signs
    for p_h in [p_low, p_high]:
        # Initial guesses
        v_grid = np.ones(y_grid_size) / y_grid_size
        u = 0.05
        y_hat = p_h
        v_h = np.sum(v_grid[y_grid >= y_hat]) * (r + delta) / (r + delta_hat)
        
        v_grid, v_h, u, y_hat = solve_given_p(out, p_h, v_grid, v_h, u, y_hat, tol_p, max_iter_p, slow_p)
        QS, QD = find_QS_QD(v_grid, v_h, u, y_hat)
        test.append(QD - QS)
        
        if verbose:
            print(f'Error for price {p_h} is {QD - QS}')
            
    if np.cumprod(test)[1] > 0:
        print('Test failed, both prices have positive/negative excess demand')
        return
    
    while err > tol and j < max_iter:
        p_h = (p_low + p_high) / 2
        
        v_grid, v_h, u, y_hat = solve_given_p(out, p_h, v_grid, v_h, u, y_hat, tol_p, max_iter_p, slow_p)
        QS, QD = find_QS_QD(v_grid, v_h, u, y_hat)
        err = np.abs(QD - QS)
        
        # If excess demand QD - QS > 0, set p_low = p_h, otherwise p_high = p_h
        if QD - QS > 0:
            p_low = p_h
        else:
            p_high = p_h
            
        if verbose and j % print_skip == 0:
            print(f'Error at iter {j} is {err}; price is {p_h}; QS is {QS}; QD is {QD}')
            
        j += 1
    
    if j < max_iter:
        print(f'Converged in {j} iterations, error is {err}; price is {p_h}; QS is {QS}; QD is {QD}')
    if j == max_iter:
        print(f'Failed to Converge. Error is {err}; price is {p_h}; QS is {QS}; QD is {QD}')
        
    return v_grid, v_h, u, y_hat, p_h

# Can check code against theoretical results by looking at spread and number of entrants
# For spread of entrants, 3 comparisons
# 1. Above y_hat, compare to y_max
# 2. Below y_hat, compare to y_max
# 3. Below y_hat, compare to y_hat (or largest y below y_hat)
# Graph results
# Keep in mind that spread_theory doesn't know that vacancies cannot be negative
def o_spread(out, v_grid, v_h, u, y_hat, p_h,
           save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\Spread'):
    
    gamma, k_grid, eta, r, delta, delta_hat = out.gamma, out.k_grid, out.eta, out.r, out.delta, out.delta_hat
    eta_h, gamma_h, k_h = out.eta_h, out.gamma_h, out.k_h
    q, y_grid = out.q, out.y_grid
    barg = out.barg
    
    theta = (np.sum(v_grid[out.y_grid < y_hat]) + v_h) / u
    
    i_m = np.argmax(y_grid[v_grid > 0])    
    i_y = np.searchsorted(y_grid, y_hat)
    
    spread_theory_1 = (c(v_grid[i_m], gamma, k_grid[i_m]) - c(v_grid, gamma, k_grid)) * (r + delta_hat)
    
    spread_theory_2 = (c(v_grid[i_m], gamma, k_grid[i_m]) * (r + delta_hat) 
                       - (c(v_grid, gamma, k_grid) / (1 - eta) - c_h(v_h, gamma_h, k_h) 
                          / (1 - eta_h)) * (r + delta) / q(theta)
                      + barg * eta * (r + delta) / ((1 - eta) * (r + delta_hat)) * np.maximum(y_grid - p_h, 0))
    
    spread_theory_3 = ((c(v_grid[i_y], gamma, k_grid[i_y]) - c(v_grid, gamma, k_grid)) * (r + delta)
                       / ((1 - eta) * q(theta))
                       - barg * eta * (r + delta) / ((1 - eta) * (r + delta_hat)) 
                       * (np.maximum(y_grid[i_y] - p_h, 0) - np.maximum(y_grid - p_h, 0)))
    
    spread_prod_y_max = y_grid[i_m] - y_grid
    spread_prod_y_max[v_grid == 0] = spread_theory_2[v_grid == 0]
    spread_prod_y_hat = y_grid[i_y] - y_grid
    spread_prod_y_hat[v_grid == 0] = spread_theory_3[v_grid == 0]
    
    dif_1 = spread_prod_y_max - spread_theory_1
    dif_2 = spread_prod_y_max - spread_theory_2
    dif_3 = spread_prod_y_hat - spread_theory_3
    
    fig, ax = plt.subplots()
    ax.plot(y_grid, spread_prod_y_max, lw=2, alpha=0.7, label='Productivity $y_{max}$')
    ax.plot(y_grid[:i_y], spread_prod_y_hat[:i_y], lw=2, alpha=0.7, label=r'Productivity $\hat{y}$')
    ax.plot(y_grid[i_y:], spread_theory_1[i_y:], lw=2, alpha=0.7, label='Theoretical 1')
    ax.plot(y_grid[:i_y], spread_theory_2[:i_y], lw=2, alpha=0.7, label='Theoretical 2')
    ax.plot(y_grid[:i_y], spread_theory_3[:i_y], lw=2, alpha=0.7, label='Theoretical 3')
    ax.set(xlabel='$y$', ylabel='Spread', title='Spread of Vacancies')
    ax.legend(loc='best')
    ax.grid()
    
    if save:
        fig.savefig(folder + file + '.pdf')
    else:
        plt.show()   
        
    fig, ax = plt.subplots()
    ax.plot(y_grid[i_y:], dif_1[i_y:], lw=2, alpha=0.7, label='Above')
    ax.plot(y_grid[:i_y], dif_2[:i_y], lw=2, alpha=0.7, label='Above-Below')
    ax.plot(y_grid[:i_y], dif_3[:i_y], lw=2, alpha=0.7, label='Below')
    ax.set(xlabel='$y$', ylabel='Spread', title='Difference in Spread of Vacancies')
    ax.legend(loc='best')
    ax.grid()
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    if save:
        fig.savefig(folder + file + '_dif.pdf')
    else:
        plt.show() 
        
# For number of vacancies, look at above and below y_hat
def o_entry(out, v_grid, v_h, u, y_hat, p_h):
    
    gamma, k_grid, eta, r, delta, delta_hat = out.gamma, out.k_grid, out.eta, out.r, out.delta, out.delta_hat
    eta_h, gamma_h, k_h = out.eta_h, out.gamma_h, out.k_h
    q, p, y_grid = out.q, out.p, out.y_grid
    barg = out.barg
    
    theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u 
    firm_sum_a = np.sum(c(v_grid[y_grid >= y_hat], gamma, k_grid[y_grid >= y_hat]) * v_grid[y_grid >= y_hat])
    firm_sum = np.sum(c(v_grid[y_grid < y_hat], gamma, k_grid[y_grid < y_hat]) * v_grid[y_grid < y_hat])
    out_sum = c_h(v_h, gamma_h, k_h) * v_h
    v_tilde = np.sum(v_grid[y_grid < y_hat])
    v_hat = np.sum(v_grid[y_grid >= y_hat])
    sum_barg = np.sum(np.maximum(y_grid[y_grid < y_hat] - p_h, 0) * v_grid[y_grid < y_hat])
    
    entry_code_a = np.sum(v_grid[y_grid >= y_hat] * (y_grid[y_grid >= y_hat] - b))
    entry_code_b = np.sum(v_grid[y_grid < y_hat] * (y_grid[y_grid < y_hat] - b))
    
    entry_theory_a = ((r + delta_hat) * firm_sum_a + eta * v_hat / (u * (1 - eta)) * firm_sum
                      + delta_hat * (r + delta + eta_h * v_h * q(theta) / u ) / ((1 - eta_h) * delta) * out_sum)
    entry_theory_b = ((r + delta + eta * q(theta) * v_tilde / u) / ((1 - eta) * q(theta)) * firm_sum
                      + eta_h * v_tilde / (u * (1 - eta_h)) * out_sum 
                      - barg * eta * (r + delta) / (r + delta_hat) * sum_barg)
    
    f.write(f'Entry above from code gives {entry_code_a:.4f}; entry from theory gives {entry_theory_a:.4f} \n')
    f.write(f'Entry below from code gives {entry_code_b:.4f}; entry from theory gives {entry_theory_b:.4f} \n')
    
    # While doing this, also check if u is consistent with LOM
    u_t = delta / (delta + p(theta))    
    f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
    # Probably related to whether p and q are < 1. Check these
    f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
       
        
# Create a pdf of wage distribution
def o_wage_dist(v_grid, v_h, y_hat, w_grid, omega,
           save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\Wage_Dist'):
    
    y_grid = out.y_grid
    
    w_plot = w_grid[(v_grid > 0) * (y_grid < y_hat)]
    v_pdf = v_grid[(v_grid > 0) * (y_grid < y_hat)] / sum(v_grid[y_grid < y_hat])
    
    fig, ax = plt.subplots()
    ax.plot(w_plot, v_pdf, alpha=0.7, color='b', label='Firms')
    ax.fill_between(w_plot, 0, v_pdf, alpha=0.1, color='b')
    ax.vlines(omega, 0, 1, alpha=0.7, color='r', label='Outsourcer')
    space = 2e-5
    ax.set_ylim(-space, max(v_pdf) + space)
    ax.set(xlabel='Wages', ylabel='Density')
    ax.legend(title="Worker's Employer", loc='best')
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    if save:
        fig.savefig(folder + file + '.pdf')
    else:
        plt.show()
        
        
#################################################################################################


# Outsourcer Planning

# Use bisection again, this time on psi, the shadow cost of using outsourcing slots

def o_p_operator_factory(out):
    "Use a baseline case out and create a function to update Lagrange multipliers and solve planner's problem"
    
    r, b, delta, gamma, k_grid = out.r, out.b, out.delta, out.gamma, out.k_grid
    delta_hat, gamma_h, k_h = out.delta_hat, out.gamma_h, out.k_h
    y_grid, y_grid_size = out.y_grid, out.y_grid_size
    y_grid_max = max(y_grid)
    q, q_p, p = out.q, out.q_p, out.p 
    
    # Find quantity supplied and quantity demanded for outsourcing
    @njit()
    def find_QS_QD_p(v_grid, v_h, u, y_hat):
        
        theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
        # Supply of outsourcing are matched outsourcers, not outsourcer vacancies
        QS = v_h * q(theta) / delta
        # Demand for outsourcing are matched firms, not firm vacancies
        QD = np.sum(v_grid[y_grid >= y_hat]) / delta_hat
        return QS, QD
    

    # Find all lagrangians through iteration     
    # update v_grid, v_h, n_grid, n_h, y_hat, lam, mu, and rho
    @njit()
    def update_grids(v_grid, v_h, u, y_hat, lam, mu, rho, psi, slow, slow_l):   
                
        v_grid_new = np.empty(y_grid_size)
                 
        theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
        
        # lam is lagrangian for hiring firms
        # mu is lagrangian for outsourcing firms
        # rho is lagrangian for outsourcer
        # psi is lagrangian for outsourcing firms taking outsourcer slots (given)
        v_sum = np.sum(v_grid[y_grid < y_hat] * lam[y_grid < y_hat]) + v_h * rho
        lam_new = (y_grid - b + (theta * q_p(theta) * v_sum / u)) / (r + delta)
        mu_new = (y_grid - (1 + r) * psi) / (r + delta_hat)
        rho_new = (-b + (1 + r) * psi + (theta * q_p(theta) * v_sum / u)) / (r + delta)
        
        lam_new = lam_new * slow_l + lam * (1 - slow_l) 
        mu_new = mu_new * slow_l + mu * (1 - slow_l)
        rho_new = rho_new * slow_l + rho * (1 - slow_l)
        
        for j in range(2e2):
            
            theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u

            v_sum = np.sum(v_grid[y_grid < y_hat] * lam_new[y_grid < y_hat]) + rho_new * v_h

            cost = lam_new[y_grid < y_hat] * q(theta) + (q_p(theta) / u) * v_sum           
            v_grid_new[y_grid < y_hat] = c_inv(cost, gamma, k_grid[y_grid < y_hat])

            v_grid_new[y_grid >= y_hat] = c_inv(mu_new[y_grid >= y_hat], gamma, k_grid[y_grid >= y_hat])

            cost_h = rho_new * q(theta) + (q_p(theta) / u) * v_sum
            v_h_new = c_h_inv(cost_h, gamma_h, k_h)

            n_grid_b = q(theta) * v_grid_new[y_grid < y_hat] / delta
            n_h = q(theta) * v_h_new / delta
            u_new = max(1 - np.sum(n_grid_b) - n_h, 1e-6)

            # Choose new y_hat by firm with closest marginal costs of outsourcing and hiring (comp near 0) 
            comp = lam_new * q(theta) + (q_p(theta) / u_new) * v_sum - mu_new
            ind_new = np.argmin(np.abs(comp))
            y_hat_new = y_grid[ind_new]

            v_grid = v_grid_new * slow + v_grid * (1 - slow) 
            v_h = v_h_new * slow + v_h * (1 - slow)
            u = u_new * slow + u * (1 - slow)
            y_hat = y_hat_new * slow + y_hat * (1 - slow)       
        
        return v_grid_new, v_h_new, u, y_hat_new, lam_new, mu_new, rho_new
    
    return update_grids, find_QS_QD_p

# Solve model given psi 
def solve_given_psi(out, psi, v_grid, v_h, u, y_hat, lam, mu, rho, tol, max_iter, slow, slow_l):
    
    update_grids, _ = o_p_operator_factory(out)
    
    # Set up initial loop parameters
    i = 0
    err = tol + 1    
    
    # Update v_grid, v_h, u, and y_hat each period
    while i < max_iter and err > tol:
        v_grid_new, v_h_new, u_new, y_hat_new, lam_new, mu_new, rho_new = update_grids(
            v_grid, v_h, u, y_hat, lam, mu, rho, psi, slow, slow_l)
        err_v = np.max(np.abs(v_grid_new - v_grid))
        err_v_h = np.abs(v_h_new - v_h)
        err_u = np.abs(u_new - u)
        err_y_hat = np.abs(y_hat_new - y_hat)
        err = max(err_v, err_v_h, err_u, err_y_hat)
        i += 1
        
        v_grid = v_grid_new
        v_h = v_h_new     
        u = u_new
        y_hat = y_hat_new   
        lam = lam_new     
        mu = mu_new
        rho = rho_new
        
    return v_grid_new, v_h_new, u_new, y_hat_new, lam_new, mu_new, rho_new


# Find psi by bisection
def find_psi(
    out, guess, tol, max_iter, psi_low, psi_high, tol_p, max_iter_p, slow_p, slow_l, verbose=True, print_skip=100
):
    
    update_grids, find_QS_QD_p = o_p_operator_factory(out) 
    
    r, b, delta, delta_hat, y_grid = out.r, out.b, out.delta, out.delta_hat, out.y_grid
    
    # Set up loop parameters    
    err = tol + 1
    j = 0
    test = []
    
    # Find QD - QS for psi_low and psi_high, make sure they have opposite signs
    for psi in [psi_low, psi_high]:
        # Initial guesses
        v_grid, v_h, u, y_hat = guess    
        lam = (y_grid - b) / (r + delta)
        mu = (y_grid - (1 + r) * psi) / (r + delta_hat)
        rho = (-b + (1 + r) * psi) / (r + delta)
        
        v_grid, v_h, u, y_hat, lam, mu, rho = solve_given_psi(
            out, psi, v_grid, v_h, u, y_hat, lam, mu, rho, tol_p, max_iter_p, slow_p, slow_l)
        QS, QD = find_QS_QD_p(v_grid, v_h, u, y_hat)
        test.append(QD - QS)
        
        if verbose:
            print(f'Error for psi {psi} is {QD - QS}')
            
    if np.cumprod(test)[1] > 0:
        print('Test failed, both prices have positive/negative excess demand')
        return
    
    
    # Reset guess
    v_grid, v_h, u, y_hat = guess    
        
    while err > tol and j < max_iter:
        psi = (psi_low + psi_high) / 2
        
        v_grid, v_h, u, y_hat, lam, mu, rho = solve_given_psi(
            out, psi, v_grid, v_h, u, y_hat, lam, mu, rho, tol, max_iter_p, slow_p, slow_l)
        QS, QD = find_QS_QD_p(v_grid, v_h, u, y_hat)
        err = np.abs(QD - QS)
        
        # If excess demand QD - QS > 0, set p_low = p_h, otherwise p_high = p_h
        if QD - QS > 0:
            psi_low = psi
        else:
            psi_high = psi
            
        if verbose and j % print_skip == 0:
            print(f'Error at iter {j} is {err}; psi is {psi}; QS is {QS}; QD is {QD}')
            
        j += 1
    
    if j < max_iter:
        print(f'Converged in {j} iterations, error is {err}; psi is {psi}; QS is {QS}; QD is {QD}')
    if j == max_iter:
        print(f'Failed to Converge. Error is {err}; psi is {psi}; QS is {QS}; QD is {QD}')
        
    return v_grid, v_h, u, y_hat


# Can check code against theoretical results by looking at spread and number of entrants
# For spread of entrants, 3 comparisons
# 1. Above y_hat, compare to y_max
# 2. Below y_hat, compare to y_max
# 3. Below y_hat, compare to y_hat (or largest y below y_hat)
# Graph results
# Keep in mind that spread_theory doesn't know that vacancies cannot be negative
def o_p_spread(out, v_grid, v_h, u, y_hat,
             save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\Planner_Spread'):
    
    gamma, k_grid, eta, r, delta, delta_hat = out.gamma, out.k_grid, out.eta, out.r, out.delta, out.delta_hat
    gamma_h, k_h = out.gamma_h, out.k_h
    q, y_grid = out.q, out.y_grid
     
    theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u 
    
    i_m = np.argmax(y_grid[v_grid > 0])    
    i_y = np.searchsorted(y_grid, y_hat)
    
    spread_theory_1 = (c(v_grid[i_m], gamma, k_grid[i_m]) - c(v_grid, gamma, k_grid)) * (r + delta_hat)
    
    spread_theory_2 = (c(v_grid[i_m], gamma, k_grid[i_m]) * (r + delta_hat) 
                       - (c(v_grid, gamma, k_grid) - c_h(v_h, gamma_h, k_h)) * (r + delta) / q(theta))
    
    spread_theory_3 = (c(v_grid[i_y], gamma, k_grid[i_y]) - c(v_grid, gamma, k_grid)) * (r + delta) / q(theta)
        
    spread_prod_y_max = y_grid[i_m] - y_grid
    spread_prod_y_max[v_grid == 0] = spread_theory_2[v_grid == 0]
    spread_prod_y_hat = y_grid[i_y] - y_grid
    spread_prod_y_hat[v_grid == 0] = spread_theory_3[v_grid == 0]
    
    dif_1 = spread_prod_y_max - spread_theory_1
    dif_2 = spread_prod_y_max - spread_theory_2
    dif_3 = spread_prod_y_hat - spread_theory_3
    
    fig, ax = plt.subplots()
    ax.plot(y_grid, spread_prod_y_max, lw=2, alpha=0.7, label='Productivity $y_{max}$')
    ax.plot(y_grid[:i_y], spread_prod_y_hat[:i_y], lw=2, alpha=0.7, label=r'Productivity $\hat{y}$')
    ax.plot(y_grid[i_y:], spread_theory_1[i_y:], lw=2, alpha=0.7, label='Theoretical 1')
    ax.plot(y_grid[:i_y], spread_theory_2[:i_y], lw=2, alpha=0.7, label='Theoretical 2')
    ax.plot(y_grid[:i_y], spread_theory_3[:i_y], lw=2, alpha=0.7, label='Theoretical 3')
    ax.set(xlabel='$y$', ylabel='Spread', title='Spread of Vacancies')
    ax.legend(loc='best')
    ax.grid()
    
    if save:
        fig.savefig(folder + file + '.pdf')
    else:
        plt.show()   
        
    fig, ax = plt.subplots()
    ax.plot(y_grid[i_y:], dif_1[i_y:], lw=2, alpha=0.7, label='Above')
    ax.plot(y_grid[:i_y], dif_2[:i_y], lw=2, alpha=0.7, label='Above-Below')
    ax.plot(y_grid[:i_y], dif_3[:i_y], lw=2, alpha=0.7, label='Below')
    ax.set(xlabel='$y$', ylabel='Spread', title='Difference in Spread of Vacancies')
    ax.legend(loc='best')
    ax.grid()
    ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    if save:
        fig.savefig(folder + file + '_dif.pdf')
    else:
        plt.show()
        
# For number of vacancies, look at above and below y_hat
def o_p_entry(out, v_grid, v_h, u, y_hat):
    
    b, gamma, k_grid, r, delta, delta_hat = out.b, out.gamma, out.k_grid, out.r, out.delta, out.delta_hat
    gamma_h, k_h = out.gamma_h, out.k_h
    q, q_p, p, y_grid = out.q, out.q_p, out.p, out.y_grid
      
    theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
    alpha = - theta * q_p(theta) / q(theta)
        
    entry_theory_a = np.sum(v_grid[y_grid >= y_hat] * (y_grid[y_grid >= y_hat] - b))
    entry_theory_b = np.sum(v_grid[y_grid < y_hat] * (y_grid[y_grid < y_hat] - b))
    
    # Find psi to estimate correct entry
    psi = (y_grid[-1] - (r + delta_hat) * c(v_grid[-1], gamma, k_grid[-1])) / (1 + r)
    
    psi_min_b = (1 + r) * psi - b
    entry_code_a = ((r + delta_hat) * np.sum(v_grid[y_grid >= y_hat] * 
                                             c(v_grid[y_grid >= y_hat], gamma, k_grid[y_grid >= y_hat]))
                    + psi_min_b * np.sum(v_grid[y_grid >= y_hat]))
    entry_code_b = (((r + delta + theta * alpha * q(theta)) / ((1 - alpha) * q(theta))) 
                    * (np.sum(v_grid[y_grid < y_hat] * c(v_grid[y_grid < y_hat], gamma, k_grid[y_grid < y_hat])) 
                       + c_h(v_h, gamma_h, k_h) * v_h) - psi_min_b * v_h)
   
    
    f.write(f'Entry above from code gives {entry_code_a:.4f}; entry from theory gives {entry_theory_a:.4f} \n')
    f.write(f'Entry below from code gives {entry_code_b:.4f}; entry from theory gives {entry_theory_b:.4f} \n')
    
    # While doing this, also check if u is consistent with LOM
    u_t = delta / (delta + p(theta))  
    f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
    # Probably related to whether p and q are < 1. Check these
    f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
########################################################################################################

# Create graph of supply and demand for outsourced workers (decentralized)
# Create graphs for various p_h

def graphing(out, p_h_min, p_h_max, p_h_grid_size, tol=1e-4, max_iter=1000, slow=1/100,
           save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\SD'):
    
    delta, eta, eta_h, y_grid, q, p = out.delta, out.eta, out.eta_h, out.y_grid, out.q, out.p
    gamma, k_grid, gamma_h, k_h = out.gamma, out.k_grid, out.gamma_h, out.k_h
    y_grid_size = out.y_grid_size
        
    p_h_grid = np.linspace(p_h_min, p_h_max, p_h_grid_size, endpoint=False)
    
    iterate, wages, get_y_hat, find_QS_QD = o_operator_factory(out)
    
    # Create grid for QS and QD
    QS_grid, QD_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Create a grid for theta, v_h, and va_grid (vacancies above y_hat)
    theta_grid, v_h_grid, v_a_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Create grid for total firms and vacancies
    v_tot_grid, n_tot_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Create grid for y_hat, omega, and val_search
    y_hat_grid, omega_grid, val_search_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Create a grid for percent of matching with outsourcer (pi) and percent firms outsourcing
    pi_grid, per_out_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Create a grid for u and theoretical u from lom
    u_grid, u_lom_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Create grids for p and q
    p_grid, q_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
    # Record the maximum price with poistive demand, start by assuming p_h_max
    i_max = int(p_h_max)
    
    
    # For each p_h, find QS and QD, which requires finding theta too
    for i, p_h in enumerate(p_h_grid):
        
        # Initial guesses
        v_grid = np.ones(y_grid_size) / y_grid_size
        u = 0.05
        y_hat = p_h
        v_h = np.sum(v_grid[y_grid >= y_hat]) * (r + delta) / (r + delta_hat)
        
        v_grid, v_h, u, y_hat = solve_given_p(out, p_h, v_grid, v_h, u, y_hat, tol, max_iter, slow)
        
        QS_grid[i], QD_grid[i] = find_QS_QD(v_grid, v_h, u, y_hat)
        w_grid, omega = wages(v_grid, v_h, u, y_hat, p_h)
        theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
        v_tot_grid[i] = np.sum(v_grid)
        n_tot_grid[i] = QD_grid[i] + np.sum(v_grid[y_grid < y_hat]) * q(theta) / delta
        v_a_grid[i] = np.sum(v_grid[y_grid >= y_hat])
        v_h_grid[i] = v_h
        y_hat_grid[i] = y_hat
        theta_grid[i] = theta
        omega_grid[i] = omega
        u_grid[i] = u
        pi_grid[i] = 100 * v_h / (np.sum(v_grid[y_grid < y_hat]) + v_h)
        per_out_grid[i] = 100 * QD_grid[i] / n_tot_grid[i]
        val_search_grid[i] = ((eta / (1 - eta) * 
                               np.sum(c(v_grid[y_grid < y_hat], gamma, k_grid[y_grid < y_hat])
                                      * v_grid[y_grid < y_hat]) 
                               + eta_h / (1 - eta_h) * c_h(v_h, gamma_h, k_h) * v_h) / u)
        p_grid[i] = p(theta)
        q_grid[i] = q(theta)
        u_lom_grid[i] = delta / (delta + p(theta))
        
        # If QD_grid[i] == 0, stop the program and record i_max
        if QD_grid[i] == 0:
            i_max = i
            break
                
    fig, ax = plt.subplots()
    ax.plot(QS_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Supply')
    ax.plot(QD_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Demand')
    ax.set(xlabel='$n$', ylabel='$p_h$', title='Supply and Demand of Outsourcing')
    ax.legend(loc='best')
    
    if save:
        fig.savefig(folder + file + '.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(QS_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Supply')
    ax.plot(v_h_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Vacancies')
    ax.set(xlabel='$n$', ylabel='$p_h$', title='Supply of Outsourcing vs Vacancies')
    ax.legend(loc='best')
    
    if save:
        fig.savefig(folder + file + '_Supply.pdf')
        plt.close()
    else:
        plt.show()
        
    fig, ax = plt.subplots()
    ax.plot(QD_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Demand')
    ax.plot(v_a_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Vacancies')
    ax.set(xlabel='$n$', ylabel='$p_h$', title='Demand for Outsourcing vs Vacancies')
    ax.legend(loc='best')
    
    if save:
        fig.savefig(folder + file + '_Demand.pdf')
        plt.close()
    else:
        plt.show()
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], QD_grid[:i_max] - QS_grid[:i_max], lw=2, alpha=0.7)
    ax.axhline(y=0, ls=':', c='k')
    ax.set(xlabel='$p_h$', ylabel=r'$QD - QS$', title='$QD - QS$ Given Price $p_h$')
    
    if save:
        fig.savefig(folder + file + '_qd_min_qs.pdf')
        plt.close()
    else:
        plt.show()  
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], pi_grid[:i_max], lw=2, alpha=0.7)
    ax.set(xlabel='$p_h$', ylabel=r'%', title='Percent Chance of Matching with Outsourcer')
    
    if save:
        fig.savefig(folder + file + '_pi.pdf')
        plt.close()
    else:
        plt.show()
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], per_out_grid[:i_max], lw=2, alpha=0.7)
    ax.set(xlabel='$p_h$', ylabel=r'%', title='Percent of Postions Outsourced')
    
    if save:
        fig.savefig(folder + file + '_per_out.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], val_search_grid[:i_max], lw=2, alpha=0.7)
    ax.set(xlabel='$p_h$', ylabel=r'Value', title='Value of Search')
    
    if save:
        fig.savefig(folder + file + '_val_search.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], theta_grid[:i_max], lw=2, alpha=0.7)
    ax.set(xlabel='$p_h$', ylabel=r'$\theta$', title='Market Tightness')
    
    
    if save:
        fig.savefig(folder + file + '_theta.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], v_tot_grid[:i_max], lw=2, alpha=0.7, label=r'Vacancies')
    ax.plot(p_h_grid[:i_max], n_tot_grid[:i_max], lw=2, alpha=0.7, label=r'Firms')
    ax.set(xlabel='$p_h$', ylabel=r'$n/v$', title='Total Firms and Vacancies')
    ax.legend(loc='best')
    
    
    if save:
        fig.savefig(folder + file + '_tot_f_v.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], p_grid[:i_max], lw=2, alpha=0.7, label=r'$p(\theta)$')
    ax.plot(p_h_grid[:i_max], q_grid[:i_max], lw=2, alpha=0.7, label=r'$q(\theta)$')
    ax.set(xlabel='$p_h$', ylabel=r'Matching Probability', title='Matching Probabilities')
    ax.legend()
    
    
    if save:
        fig.savefig(folder + file + '_match_prob.pdf')
        plt.close()
    else:
        plt.show() 
        
        
    y_min = min(out.y_grid)
    y_max = max(out.y_grid)
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], y_hat_grid[:i_max], lw=2, alpha=0.7)
    ax.axhline(y=y_min, ls=':', c='k')
    ax.axhline(y=y_max, ls=':', c='k')
    ax.set(xlabel='$p_h$', ylabel='$y^*$', title='$y^*$ Given Price $p_h$')
    
    if save:
        fig.savefig(folder + file + '_y_hat.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], omega_grid[:i_max], lw=2, alpha=0.7)
    ax.set(xlabel='$p_h$', ylabel='$\omega$', title='Wage Given Price $p_h$')
    
    if save:
        fig.savefig(folder + file + '_omega.pdf')
        plt.close()
    else:
        plt.show() 
        
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], u_grid[:i_max], lw=2, alpha=0.7, label='u_model')    
    ax.set(xlabel='$p_h$', ylabel='$u$', title='Unemployment Given Price $p_h$')
    
    if save:
        fig.savefig(folder + file + '_u.pdf')
        plt.close()
    else:
        plt.show()      
    
    fig, ax = plt.subplots()
    ax.plot(p_h_grid[:i_max], u_grid[:i_max], lw=2, alpha=0.7, label='u_model')    
    ax.plot(p_h_grid[:i_max], u_lom_grid[:i_max], lw=2, alpha=0.7, label='u_lom')
    ax.set(xlabel='$p_h$', ylabel='$u$', title='Unemployment Given Price $p_h$')
    ax.legend()
    
    if save:
        fig.savefig(folder + file + '_u_check.pdf')
        plt.close()
    else:
        plt.show()
    
    
#######################################################################################
        
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Run code

B = 0.95
r = 1 / B - 1
delta = 0.03
eta = 0.5
eta_h = 0.25
phi = 5e-1 
y_min = 5
y_max = 10
b = y_min * .4 
y_grid_size = 4000
gamma = 2
delta_hat = 0.01
gamma_h = 2
k_grid = np.logspace(5, 7, y_grid_size)
k_h = 5e2
barg = 0

f.write(f"""In this file, we have parameters: \n
B = {B} \n
r = {r} \n
delta = {delta} \n
delta_hat = {delta_hat} \n 
eta = {eta} \n
eta_h = {eta_h} \n
phi = {phi} \n 
y_min = {y_min} \n 
y_max = {y_max} \n
y_grid_size = {y_grid_size} \n
b = {b} \n 
gamma = {gamma} \n 
gamma_h = {gamma_h} \n 
k_grid = np.logspace(5, 7, y_grid_size) \n 
k_h = {k_h} \n
barg = {barg} \n""")

bl = BaselineDiscrete(B, b, delta, eta, phi, y_min, y_max, y_grid_size, gamma, k_grid)
out = OutsourcerDiscrete(
    B, b, delta, delta_hat, eta, eta_h, phi, y_min, y_max, y_grid_size,
    gamma, gamma_h, k_grid, k_h, barg
)

q, y_grid = bl.q, bl.y_grid

###########################################
f.write(" \n For Baseline; Decentralized Case \n")

tol_1 = 1e-8
max_iter_1 = 1e5
slow_1 = 1e-2
print_skip = 1e4
v_grid_b, u_b = solve_model(bl, tol=tol_1, verbose=True, print_skip=print_skip, max_iter=max_iter_1, slow=slow_1)

f.write(f"""
tol_1 = {tol_1} \n
max_iter_1 = {max_iter_1} \n
slow_1 = {slow_1} \n""")

iterate, wages = b_operator_factory(bl)
theta_b = np.sum(v_grid_b) / u_b
w_grid_b, theta_b = wages(v_grid_b, u_b)

n_grid_b = q(theta_b) * v_grid_b / delta

b_spread(bl, v_grid_b, u_b, save=True, folder=folder, file=r'\baseline_spread')
b_entry(bl, v_grid_b, u_b)
b_wage_dist(v_grid_b, w_grid_b, save=True, folder=folder, file=r'\wage_dist')

#######################################
f.write(" \n For Baseline; Planner's Case \n")

guess = [v_grid_b, u_b]
tol_2 = 1e-8
max_iter_2 = 1e5
slow_2 = 1e-2
ps_2 = 1e4

f.write(f"""
tol_2 = {tol_2} \n
max_iter_2 = {max_iter_2} \n
slow_2 = {slow_2} \n""")

v_grid_p, u_p = solve_p_model(bl, guess, tol_2, max_iter_2, slow_2, verbose=True, print_skip=ps_2)

theta_p = np.sum(v_grid_p) / u_p
n_grid_p = v_grid_p * q(theta_p) / delta

p_spread(bl, v_grid_p, u_p, save=True, folder=folder, file=r'\baseline_p_spread')
p_entry(bl, v_grid_p, u_p)

###########################################
f.write(" \n For Outsourcers; Decentralized Case \n")

tol_3 = 1e-8
max_iter_3 = 3.5e1
tol_p_3 = 1e-8
max_iter_p_3 = 1e4
slow_p_3 = 1e-2
print_skip_3 = 5

p_low = 0
p_high = y_max + 1

f.write(f"""
tol_3 = {tol_3} \n
max_iter_3 = {max_iter_3} \n
tol_p_3 = {tol_p_3} \n
max_iter_p_3 = {max_iter_p_3} \n
slow_p_3 = {slow_p_3} \n
p_low = {p_low} \n
p_high = {p_high} \n""")

v_grid_o, v_h_o, u_o, y_hat_o, p_h_o = find_p(
    out, tol_3, max_iter_3, p_low, p_high, tol_p_3, max_iter_p_3, slow_p_3, True, print_skip_3
)

iterate, wages, get_y_hat, find_QS_QD = o_operator_factory(out)

theta_o = (np.sum(v_grid_o[y_grid < y_hat_o]) + v_h_o) / u_o
w_grid_o, omega_o = wages(v_grid_o, v_h_o, u_o, y_hat_o, p_h_o)

n_grid_o = np.empty(y_grid_size)
n_grid_o[y_grid < y_hat_o] = q(theta_o) * v_grid_o[y_grid < y_hat_o] / delta
n_grid_o[y_grid >= y_hat_o] = v_grid_o[y_grid >= y_hat_o] / delta_hat
n_h_o = q(theta_o) * v_h_o / delta

o_spread(out, v_grid_o, v_h_o, u_o, y_hat_o, p_h_o,
           save=True, folder=folder, file=r'\outsourcer_spread')

o_entry(out, v_grid_o, v_h_o, u_o, y_hat_o, p_h_o)

o_wage_dist(v_grid_o, v_h_o, y_hat_o, w_grid_o, omega_o, 
           save=True, folder=folder, file=r'\outsourcer_wage_dist')


# What percent of firms outsource?
per_out_o = n_h_o / np.sum(n_grid_o) * 100
f.write(f'Decentralized percent of matches outsourcing is {per_out_o:2.0f} \n')

###########################################
f.write(" \n For Outsourcers; Planner's Case \n")

tol_4 = 1e-8
max_iter_4 = 3.5e1
tol_p_4 = 1e-8
max_iter_p_4 = 1e4
slow_p_4 = 1e-2
slow_l_4 = 5e-2
ps_4 = 5

psi_low = p_h_o - 2
psi_high = min(p_h_o + 2, y_max)

f.write(f"""
tol_4 = {tol_4} \n
max_iter_4 = {max_iter_4} \n
tol_p_4 = {tol_p_4} \n
max_iter_p_4 = {max_iter_p_4} \n
slow_p_4 = {slow_p_4} \n
slow_l_4 = {slow_l_4} \n
psi_low = {psi_low} \n
psi_high = {psi_high} \n""")

guess_o = [v_grid_o, v_h_o, u_o, y_hat_o]

v_grid_o_p, v_h_o_p, u_o_p, y_hat_o_p = find_psi(
    out, guess_o, tol_4, max_iter_4, psi_low, psi_high,
    tol_p_4, max_iter_p_4, slow_p_4, slow_l_4, verbose=True, print_skip=ps_4
)

theta_o_p = (np.sum(v_grid_o_p[y_grid < y_hat_o_p]) + v_h_o_p) / u_o_p

n_grid_o_p = np.empty(y_grid_size)
n_grid_o_p[y_grid >= y_hat_o_p] = v_grid_o_p[y_grid >= y_hat_o_p] / delta_hat
n_grid_o_p[y_grid < y_hat_o_p] = v_grid_o_p[y_grid < y_hat_o_p] * q(theta_o_p) / delta
n_h_o_p = v_h_o_p * q(theta_o_p) / delta

o_p_spread(out, v_grid_o_p, v_h_o_p, u_o_p, y_hat_o_p,
           save=True, folder=folder, file=r'\outsourcer_p_spread')

o_p_entry(out, v_grid_o_p, v_h_o_p, u_o_p, y_hat_o_p)

o_wage_dist(v_grid_o, v_h_o, y_hat_o, w_grid_o, omega_o,
           save=True, folder=folder, file=r'\outsourcer_wage_dist')

# What percent of firms outsource?
per_out_o_p = n_h_o_p / np.sum(n_grid_o_p) * 100
f.write(f'Planner percent of matches outsourcing is {per_out_o_p:2.0f} \n')

###########################################
f.write(" \n For Outsourcers; Supply and Demand \n")

p_h_min = y_min
p_h_max = y_max
p_h_grid_size = 100
tol_5 = 1e-8
max_iter_5 = 1e4
slow_5 = 1e-2

f.write(f"""
tol_5 = {tol_5} \n
max_iter_5 = {max_iter_5} \n
p_h_grid_size = {p_h_grid_size} \n
p_h_min = {p_h_min} \n
p_h_max = {p_h_max} \n
slow_5 = {slow_5} \n""")

graphing(out, p_h_min, p_h_max, p_h_grid_size, tol_5, max_iter_5, slow_5,
              save=True, folder=folder, file=r'\outsourcer_SD')


#########################################

# Compare the welfare of baseline and outsourcer, print results
w_welfare_b = np.sum(n_grid_b * w_grid_b) + u_b * b
f_welfare_b = np.sum(n_grid_b * (bl.y_grid - w_grid_b)) - np.sum(C(v_grid_b, gamma, k_grid))
t_welfare_b = np.sum(n_grid_b * bl.y_grid) + u_b * b - np.sum(C(v_grid_b, gamma, k_grid))

welfare_p = np.sum(n_grid_p * bl.y_grid) + u_p * b - np.sum(C(v_grid_p, gamma, k_grid))
per_tot_w_b = t_welfare_b / welfare_p * 100

w_welfare_o = np.sum(n_grid_o[out.y_grid < y_hat_o] * w_grid_o[out.y_grid < y_hat_o]) + n_h_o * omega_o + u_o * b
firm_prof_h = n_grid_o * (out.y_grid - w_grid_o)
firm_prof_o = n_grid_o * (out.y_grid - p_h_o)
f_welfare_o = (np.sum(firm_prof_h[out.y_grid < y_hat_o]) + np.sum(firm_prof_o[out.y_grid >= y_hat_o])  
               - np.sum(C(v_grid_o, gamma, k_grid)))
o_welfare_o = n_h_o * (p_h_o - omega_o) - C_h(v_h_o, gamma_h, k_h)
t_welfare_o = np.sum(n_grid_o * out.y_grid) - np.sum(C(v_grid_o, gamma, k_grid)) - C_h(v_h_o, gamma_h, k_h)

welfare_o_p = (np.sum(n_grid_o_p * out.y_grid) + u_o_p * b 
               - np.sum(C(v_grid_o_p, gamma, k_grid)) - C_h(v_h_o_p, gamma_h, k_h))
per_tot_w_o = t_welfare_o / welfare_o_p * 100

f.write("Comparing Welfare \n")
f.write(f"""For the baseline decentralized case \n
worker welfare is {w_welfare_b:.4f} \n
firm welfare is {f_welfare_b:.4f} \n 
total welfare is {t_welfare_b:.4f} \n
For the baseline planner's problem, total welfare is {welfare_p:.4f} \n
The decentralized solution captures {per_tot_w_b:2.0f} percent of efficient welfare \n
For the outsourcer's decentralized case \n
worker welfare is {w_welfare_o:.4f} \n
firm welfare is {f_welfare_o:.4f} \n 
outsourcer welfare is {o_welfare_o:.4f} \n 
total welfare is {t_welfare_o:.4f} \n
For the baseline planner's problem, total welfare is {welfare_o_p:.4f} \n
The decentralized solution captures {per_tot_w_o:2.0f} percent of efficient welfare \n
""")


f.close()

#############################################
########################

# Now graph figures of interest

# Wages, Vacancies, and Matches in baseline decentralized
fig, ax = plt.subplots()
ax.plot(bl.y_grid, w_grid_b, lw=2, alpha=0.7)
ax.set(xlabel="Firm Productivity $y$", ylabel="Wage $w$")

fig.savefig(folder + r'\baseline_wages.pdf')


fig, ax = plt.subplots()
ax.plot(bl.y_grid, v_grid_b, lw=2, alpha=0.7)
ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

fig.savefig(folder + r'\baseline_vacancies.pdf')


fig, ax = plt.subplots()
ax.plot(bl.y_grid, n_grid_b, lw=2, alpha=0.7)
ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

fig.savefig(folder + r'\baseline_matches.pdf')

plt.close(fig='all')

# Vacancies and matches in baseline decentralized vs planner's

fig, ax = plt.subplots()
ax.plot(bl.y_grid, v_grid_b, lw=2, alpha=0.7, label='Decentralized')
ax.plot(bl.y_grid, v_grid_p, lw=2, alpha=0.7, label="Planner's")
ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_d_p_vacancies.pdf')



fig, ax = plt.subplots()
ax.plot(bl.y_grid, n_grid_b, lw=2, alpha=0.7, label='Decentralized')
ax.plot(bl.y_grid, n_grid_p, lw=2, alpha=0.7, label="Planner's")
ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_d_p_matches.pdf')

plt.close(fig='all')

# Wages, vacancies, and matches for outsourcers decentralized

fig, ax = plt.subplots()
omega_grid_o = omega_o * np.ones(y_grid_size)
p_h_grid_o = p_h_o * np.ones(y_grid_size)
fig, ax = plt.subplots(figsize=(6,6))
ax.plot(out.y_grid[out.y_grid < y_hat_o], w_grid_o[out.y_grid < y_hat_o], lw=2, alpha=0.7, c='b')
ax.plot(out.y_grid[out.y_grid >= y_hat_o], w_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='b', linestyle=':')
ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':', alpha=0.7)
ax.plot(out.y_grid[out.y_grid >= y_hat_o], omega_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='b')
ax.plot(out.y_grid[out.y_grid >= y_hat_o], p_h_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='k')
ax.set(xlabel="Firm Productivity $y$", ylabel="Wages $w$")

fig.savefig(folder + r'\outsourcer_wages.pdf')


fig, ax = plt.subplots()
ax.plot(out.y_grid, v_grid_o, lw=2, alpha=0.7)
ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

fig.savefig(folder + r'\outsourcer_vacancies.pdf')


fig, ax = plt.subplots()
ax.plot(out.y_grid, n_grid_o, lw=2, alpha=0.7)
ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

fig.savefig(folder + r'\outsourcer_matches.pdf')

plt.close(fig='all')

# Vacancies and matches for outsourcers decentralized vs planner's
fig, ax = plt.subplots()
ax.plot(out.y_grid, v_grid_o, lw=2, alpha=0.7, label="Decentralized", color='blue')
ax.plot(out.y_grid, v_grid_o_p, lw=2, alpha=0.7, label="Planner's", color='orange')
ax.axvline(x=y_hat_o, c='green', lw=1, linestyle=':')
ax.axvline(x=y_hat_o_p, c='red', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\outsourcer_d_p_vacancies.pdf')


fig, ax = plt.subplots()
ax.plot(out.y_grid, n_grid_o, lw=2, alpha=0.7, label="Decentralized", color='blue')
ax.plot(out.y_grid, n_grid_o_p, lw=2, alpha=0.7, label="Planner's", color='orange')
ax.axvline(x=y_hat_o, c='green', lw=1, linestyle=':')
ax.axvline(x=y_hat_o_p, c='red', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\outsourcer_d_p_matches.pdf')

plt.close(fig='all')

# Wages, vacancies, and matches for baseline vs outsourcers decentralized

fig, ax = plt.subplots()
ax.plot(bl.y_grid, w_grid_b, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid[out.y_grid < y_hat_o], w_grid_o[out.y_grid < y_hat_o], lw=2, alpha=0.7, c='orange', label='Outsourcers')
ax.plot(out.y_grid[out.y_grid >= y_hat_o], w_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='orange', linestyle=':')
ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':', alpha=0.7)
ax.plot(out.y_grid[out.y_grid >= y_hat_o], omega_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='orange')
ax.plot(out.y_grid[out.y_grid >= y_hat_o], p_h_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='k')
ax.set(xlabel="Firm Productivity $y$", ylabel="Wages $w$")
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_v_outsourcer_wages.pdf')


fig, ax = plt.subplots()
ax.plot(bl.y_grid, v_grid_b, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid, v_grid_o, lw=2, alpha=0.7, label="Outsourcers")
ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_v_outsourcer_vacancies.pdf')


fig, ax = plt.subplots()
ax.plot(bl.y_grid, n_grid_b, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid, n_grid_o, lw=2, alpha=0.7, label="Outsourcers")
ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_v_outsourcer_matches.pdf')

plt.close(fig='all')

# Vacancies and matches for baseline vs outsourcers planner's

fig, ax = plt.subplots()
ax.plot(bl.y_grid, v_grid_p, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid, v_grid_o_p, lw=2, alpha=0.7, label="Outsourcers")
ax.axvline(x=y_hat_o_p, c='k', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_v_outsourcer_p_vacancies.pdf')


fig, ax = plt.subplots()
ax.plot(bl.y_grid, n_grid_p, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid, n_grid_o_p, lw=2, alpha=0.7, label="Outsourcers")
ax.axvline(x=y_hat_o_p, c='k', lw=1, linestyle=':')
ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_v_outsourcer_p_matches.pdf')

plt.close(fig='all')

# Plot wage distribution for no outsourcing vs oustsourcing cases

w_plot_b = w_grid_b[v_grid_b > 0]
v_pdf_b = v_grid_b[v_grid_b > 0] / sum(v_grid_b)

w_plot_o = w_grid_o[(v_grid_o > 0) * (y_grid < y_hat_o)]
v_pdf_o = v_grid_o[(v_grid_o > 0) * (y_grid < y_hat_o)] / sum(v_grid_o[y_grid < y_hat_o])

fig, ax = plt.subplots()
ax.plot(w_plot_b, v_pdf_b, alpha=0.7, color='c', label='Baseline (Firms)')
ax.fill_between(w_plot_b, 0, v_pdf_b, alpha=0.1, color='c')
ax.plot(w_plot_o, v_pdf_o, alpha=0.7, color='b', label='Outsourcer (Firms)')
ax.fill_between(w_plot_o, 0, v_pdf_o, alpha=0.1, color='b')
ax.vlines(omega_o, 0, 1, alpha=0.7, color='r', label='Outsourcer (Outsourcer)')
space = 2e-5
ax.set_ylim(-space, max(max(v_pdf_b), max(v_pdf_o)) + space)
ax.set(xlabel='Wages', ylabel='Density')
ax.legend(title="Model (Employer)", loc='best')
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

fig.savefig(folder + r'\baseline_v_outsourcer_wage_dist.pdf')

plt.close(fig='all')


# Use differemces not ratios as goes near 0

v_grid_dif_b = v_grid_b - v_grid_p
v_grid_dif_o = v_grid_o - v_grid_o_p

n_grid_dif_b = n_grid_b - n_grid_p
n_grid_dif_o = n_grid_o - n_grid_o_p 

fig, ax = plt.subplots()
ax.plot(bl.y_grid, v_grid_dif_b, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid, v_grid_dif_o, lw=2, alpha=0.7, label="Outsourcers")
ax.set(xlabel="Firm Productivity $y$", ylabel="Difference")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_outsourcer_d_p_dif_vacancies.pdf')


fig, ax = plt.subplots()
ax.plot(bl.y_grid, n_grid_dif_b, lw=2, alpha=0.7, label="Baseline")
ax.plot(out.y_grid, n_grid_dif_o, lw=2, alpha=0.7, label="Outsourcers")
ax.set(xlabel="Firm Productivity $y$", ylabel="Difference")
ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
ax.legend(loc='best')

fig.savefig(folder + r'\baseline_outsourcer_d_p_dif_matches.pdf')

plt.close(fig='all')

Converged in 2408 iterations
Converged in 29 iterations


C:\Users\sps55207\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:358: RuntimeWarning: invalid value encountered in double_scalars


Error for price 0 is 41.44049095300378
Error for price 11 is -0.7926055940639519
Error at iter 0 is 3.4344198068624454; price is 5.5; QS is 0.3131554554781562; QD is 3.7475752623406016
Error at iter 5 is 0.0468349168830578; price is 7.390625; QS is 0.428584165066285; QD is 0.3817492481832272
Error at iter 10 is 0.0026782845582198345; price is 7.32080078125; QS is 0.4215241618025516; QD is 0.4242024463607714
Error at iter 15 is 0.0002717944553228824; price is 7.3243255615234375; QS is 0.42188688363235055; QD is 0.42215867808767343
Error at iter 20 is 0.00026979384730085876; price is 7.324330806732178; QS is 0.42188751542512876; QD is 0.4221573092724296
Error at iter 25 is 0.00026810584044595354; price is 7.324335232377052; QS is 0.42188804849413425; QD is 0.4221561543345802
Error at iter 30 is 8.329014588531747e-05; price is 7.324335360433906; QS is 0.4218664367215336; QD is 0.4217831465756483
Failed to Converge. Error is 8.330275134443843e-05; price is 7.324335358192911; QS is 0.421866

In [6]:
%debug

> c:\users\sps55207\appdata\local\continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py(56)_wrapfunc()
     54 def _wrapfunc(obj, method, *args, **kwds):
     55     try:
---> 56         return getattr(obj, method)(*args, **kwds)
     57 
     58     # An AttributeError occurs if the object does not have

ipdb> u
> c:\users\sps55207\appdata\local\continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py(1103)argmax()
   1101 
   1102     """
-> 1103     return _wrapfunc(a, 'argmax', axis=axis, out=out)
   1104 
   1105 

ipdb> u
> <ipython-input-4-7862f27dbc78>(305)p_spread()
    303 
    304     # Find highest positive entry
--> 305     i_m = np.argmax(y_grid[v_grid > 0])
    306 
    307     spread_prod = (c(v_grid[-1], gamma, k_grid[-1]) - c(v_grid, gamma, k_grid)) * (r + delta) / q(theta)

ipdb> v_grid
array([0., 0., 0., ..., 0., 0., 0.])
ipdb> y_grid
array([ 5.        ,  5.00125031,  5.00250063, ...,  9.99749937,
        9.99874969, 10.        ])
ipdb> c


In [5]:
bl

In [1]:
# # Old version of code
# # Run Baseline and Outsourcers all together

# folder = r"C:\Users\sps55207\Desktop\Outsourcing\Figures"
# file_ = r"\Output.txt"
# f = open(folder + file_, "w")

# # Baseline Decentralized

# import numpy as np
# import matplotlib.pyplot as plt
# import quantecon as qe
# from numba import njit, prange, vectorize
# from scipy import optimize
# from IPython.core.debugger import set_trace

# # Want to vectorize cost of vacancy, marginal cost of vacancy, and inverse marginal cost of vacancy

# # See if we can raise q(theta)k
# @vectorize
# def C(v, gamma=2, k=1):
#     return k * v**gamma

# @vectorize
# def c(v, gamma=2, k=1):
#     if v > 0:
#         return k * gamma * v**(gamma - 1)
#     else:
#         return 0
     
# @vectorize
# def c_inv(c, gamma=2, k=1):
#     if c > 0:
#         return (c / gamma / k)**(1 / (gamma - 1))
#     else:
#         return 0
    
# #############################
    
# # Set up the baseline environment, starting with a discrete model
# # For now, have entry costs be exponential, matching probability be q=phi * theta**(1 / 2) 
# class BaselineDiscrete:
#     """
#     B is discount rate beta
#     r is interest rate defined by beta
#     b is home production
#     delta is job loss rate
#     eta is worker bargaining power    
#     phi is effectiveness of matching function
#     q is firm's matching function, worker's is p=theta*q(theta)
#     y_min, y_max, y_grid_size determine productivity grid
#     gamma is steepness of marginal cost
#     q is matching function of firms
#     p is matching function of workers
#     f is for printing
#     """
#     def __init__(self, B, b, delta, eta, phi, y_min, y_max, y_grid_size, gamma, k, f):
        
#         self.B, self.b, self.delta, self.eta, self.gamma, self.k = B, b, delta, eta, gamma, k
#         self.r = 1 / B - 1
        
#         self.y_grid = np.linspace(y_min, y_max, y_grid_size)
#         self.y_grid_size = y_grid_size        
        
#         self.q = njit(lambda x: min(phi * x**(-1 / 2), 1))
#         self.p = njit(lambda x: min(phi * x**(1 / 2), 1))
#         self.q_p = njit(lambda x: -phi / 2 * x**(-3 / 2))
        
#         self.f = f
        
        
# # Set up functions of problem
# def operator_factory(bl):
#     "Use a baseline case bl and create a function to find next vacancy and unemployment iterations"
    
#     B, r, b, delta, eta, gamma, k = bl.B, bl.r, bl.b, bl.delta, bl.eta, bl.gamma, bl.k
#     y_grid, y_grid_size = bl.y_grid, bl.y_grid_size
#     q, p = bl.q, bl.p
#     f = bl.f
    
#     # Use v_grid and u to find theta and f_grid (make sure v not 0)
#     @njit()
#     def get_dist(v_grid, u):
#         v = max(np.sum(v_grid), 1e-8)
#         f_grid = v_grid / v
#         theta = v / u
#         return theta, f_grid
    
#     # Find wages for each postion
#     @njit()
#     def wages(v_grid, theta, f_grid):
#         return b + eta * (y_grid - b + theta * np.sum(c(v_grid, gamma, k) * f_grid))
    
#     # Find next v_grid and u
#     @njit()
#     def iterate(v_grid, u):  

#         theta, f_grid = get_dist(v_grid, u)

#         w_grid = wages(v_grid, theta, f_grid)

#         # Use w_grid to find out how many of firm i wants to enter (cannot be below 0)
#         v_grid_new = c_inv(q(theta) * (y_grid - w_grid) / (r + delta), gamma, k)
        
# #         # Update u_new using steady state rule
# #         u_new = delta / (delta + p(theta))

#         # Try updating u_new using unemployed workers
#         n_grid_new = q(theta) * v_grid_new / delta
#         u_new = max(1 - np.sum(n_grid_new), 1e-5)
        
#         return v_grid_new, u_new 
    
#     return iterate, get_dist, wages

# def solve_model(bl, tol=1e-4, max_iter=1000, verbose=True, print_skip=25, slow=1/100):
    
#     iterate, get_dist, wages = operator_factory(bl)
    
#     # Set up initial guesses and loop parameters
#     v_grid = np.ones(bl.y_grid_size) / bl.y_grid_size
#     u = 0.05
#     i = 0
#     err = tol + 1    
    
#     # Update v_grid and u each period
#     while i < max_iter and err > tol:
#         v_grid_new, u_new = iterate(v_grid, u)
#         err_v = np.max(np.abs(v_grid_new - v_grid))
#         err_u = np.abs(u_new - u)
#         err = max(err_v, err_u)
#         i += 1
        
#         if verbose and i % print_skip == 0:
#             print(f'Error at iter {i} is {err}')
# #             theta, f_grid = get_dist(v_grid_new, u_new)
# #             f.write(u, theta, bl.q(theta), bl.p(theta), delta / (delta + bl.p(theta)))
            
# #         if verbose and i % print_skip == 1:
# #             f.write(f'Error at iter {i} is {err}')
# #             theta, f_grid = get_dist(v_grid_new, u_new)
# #             f.write(u, theta, bl.q(theta), bl.p(theta), delta / (delta + bl.p(theta)))  
        
# #         if verbose and i >= 0 and i <= 5:
# #             f.write(f'Error at iter {i} is {err}')
# #             theta, f_grid = get_dist(v_grid_new, u_new)
# #             f.write(u, theta, bl.q(theta), bl.p(theta), delta / (delta + bl.p(theta)))  
# #             f.write(u_new, v_grid_new) 
        
#         v_grid, u = slow * v_grid_new + (1 - slow) * v_grid, slow * u_new + (1 - slow) * u
        
#     if i == max_iter:
#         print("Failed to Converge")
        
#     if verbose and i < max_iter:
#         print(f'Converged in {i} iterations')
    
#     return v_grid_new, u_new

# # Can check code against theoretical results by looking at spread and number of entrants
# # For spread of entrants, compare each firms entry to highest. Graph results
# # Keep in mind that spread_theory doesn't know that vacancies cannot be negative
# def spread(bl, v_grid, u,
#            save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Baseline\Spread'):
    
#     gamma, k, eta, r, delta = bl.gamma, bl.k, bl.eta, bl.r, bl.delta
#     q, y_grid = bl.q, bl.y_grid
#     f = bl.f
    
#     iterate, get_dist, wages = operator_factory(bl)
#     theta, f_grid = get_dist(v_grid, u)
    
#     spread_prod = (c(v_grid[-1], gamma, k) - c(v_grid, gamma, k)) * (r + delta) / ((1 - eta) * q(theta))
#     spread_theory = (y_grid[-1] - y_grid) 
#     spread_theory[v_grid == 0] = spread_prod[v_grid == 0]
#     dif = spread_theory - spread_prod
    
#     fig, ax = plt.subplots()
#     ax.plot(y_grid, spread_prod, lw=2, alpha=0.7, label='Production')
#     ax.plot(y_grid, spread_theory, lw=2, alpha=0.7, label='Theoretical')
#     ax.set(xlabel='$y$', ylabel='Spread')
#     ax.legend(loc='best')
    
    
#     if save:
#         fig.savefig(folder + file + '.pdf')
#     else:
#         plt.show()   
        
#     fig, ax = plt.subplots()
#     ax.plot(y_grid, dif, lw=2, alpha=0.7)
#     ax.set(xlabel='$y$', ylabel='Spread')
#     ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    
#     if save:
#         fig.savefig(folder + file + '_dif.pdf')
#     else:
#         plt.show()  
        
#     plt.close(fig='all')

# # For number of vacancies, just have two numbers to compare
# def entry(bl, v_grid, u):
    
#     b, gamma, k, eta, r, delta = bl.b, bl.gamma, bl.k, bl.eta, bl.r, bl.delta
#     q, p, y_grid = bl.q, bl.p, bl.y_grid
#     f = bl.f
    
#     iterate, get_dist, wages = operator_factory(bl)
#     theta, f_grid = get_dist(v_grid, u)
    
#     entry_theory = np.sum(v_grid * (y_grid - b))
#     entry_code = (r + delta + eta * theta * q(theta)) * np.sum(v_grid * c(v_grid, gamma, k)) / ((1 - eta) * q(theta))
    
#     f.write(f'Entry from code gives {entry_code:.4f}; Entry from theory gives {entry_theory:.4f} \n')
    
#     # While doing this, also check if u is consistent with LOM
#     u_t = delta / (delta + p(theta))    
#     f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
#     # Probably related to whether p and q are < 1. Check these
#     f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
# ############################################################################################################

# # Baseline PLanner's
# # Try to calculate shadow cost of a vacancy for every type and use this to determine optimal amount

# def p_operator_factor(bl):
#     "Use a baseline case bl and create a function to update Lagrange multipliers and solve planner's problem"
    
#     B, r, b, delta, eta, gamma = bl.B, bl.r, bl.b, bl.delta, bl.eta, bl.gamma
#     y_grid, y_grid_size = bl.y_grid, bl.y_grid_size
#     q, q_p, p = bl.q, bl.q_p, bl.p
#     f = bl.f
    
#     # Use v_grid and n_grid to find u and theta
#     @njit()
#     def p_get_dist(v_grid, n_grid):
#         u = max(1 - np.sum(n_grid), 1e-6)
#         theta = max(np.sum(v_grid), 1e-6) / u
#         return u, theta
    
#     # lam is the Lagrangian on each firm. Needs to be estimated each time    
#     # Find all lam through iteration
#     @njit()
#     def find_lam(v_grid, u, theta, lam, tol_l=1e-4, max_iter_l=1000, slow_l=1e-2):       
#         i = 0
#         err = tol_l + 1
# #         lam = np.ones(y_grid_size)
#         while i < max_iter_l and err > tol_l:
#             lam_new = (y_grid - b + (theta * q_p(theta) / u) * np.sum(v_grid * lam)) / (r + delta)
#             err = np.max(np.abs(lam_new - lam))
#             i += 1
#             lam = slow_l * lam_new + (1 - slow_l) * lam
            
#         if i == max_iter_l:
#             print('lagrangian failed to converge')
        
#         return lam_new
    
#     # update v_grid and n_grid
#     @njit()
#     def update_grids(v_grid, n_grid, lam, tol_l, max_iter_l, slow=1, slow_l=1e-2):
        
#         u, theta = p_get_dist(v_grid, n_grid)
        
#         lam_new = find_lam(v_grid, u, theta, lam, tol_l, max_iter_l, slow_l)
        
#         # Given a lam_new, repeatedly update v_grid and n_grid hopefully will be faster
#         for j in range(1 / slow):        
#             v_grid_new = c_inv(lam_new * q(theta) + ((q_p(theta) / u) * np.sum(v_grid * lam_new)), gamma, k)
            
#             n_grid_new = q(theta) * v_grid_new / delta
            
#             v_grid = v_grid_new * slow + v_grid * (1 - slow)            
#             n_grid = n_grid_new * slow + n_grid * (1 - slow)

#             u, theta = p_get_dist(v_grid, n_grid)        
        
#         return v_grid_new, n_grid_new, lam_new
    
#     return update_grids, p_get_dist, find_lam
        
# # Solve model
# def solve_p_model(bl, guess, tol_l=1e-4, tol=1e-4, max_iter_l=1000, max_iter=10000, verbose=True, print_skip=25, 
#                    slow=1, slow_l=1e-2):   

#     update_grids, _, _ = p_operator_factor(bl)
#     f = bl.f

#     # Set up initial guesses and loop parameters
# #     # Use intitial guess from decentralized problem
# #     v_grid, u = solve_model(bl, tol=1e-8, verbose=False, print_skip=1000, max_iter=100000, slow=1/1000)
#     v_grid, u = guess
#     theta = np.sum(v_grid) / u
#     n_grid = (bl.q(theta) * v_grid) / bl.delta
    
#     # Carry around lam to make re-estimating faster
#     lam = (bl.y_grid - bl.b) / (bl.r + bl.delta)
    
#     i = 0
#     err = tol + 1 
    
#     # Update v_grid, n_grid, and lamda each period
#     while i < max_iter and err > tol:
#         v_grid_new, n_grid_new, lam_new = update_grids(v_grid, n_grid, lam, tol_l, max_iter_l, slow, slow_l)
#         err_v = np.max(np.abs(v_grid_new - v_grid))
#         err_n = np.max(np.abs(n_grid_new - n_grid))
#         err = max(err_v, err_n)
#         i += 1
        
#         if verbose and i % print_skip == 0:
#             print(f'Error at iter {i} is {err}')
#             print(v_grid[-1])
            
#         if verbose and i % print_skip == 1:
#             print(f'Error at iter {i} is {err}')
#             print(v_grid[-1]) 
        
# #         if verbose and i >= 0 and i <= 5:
# #             print(f'Error at iter {i} is {err}')
# #             print(v_grid[-1])
        
#         v_grid, n_grid = v_grid_new * slow + v_grid * (1 - slow), n_grid_new * slow + n_grid * (1 - slow)
#         lam = lam_new * slow + lam * (1 - slow)
        
        
#     if i == max_iter:
#         print("Failed to Converge")
        
#     if verbose and i < max_iter:
#         print(f'Converged in {i} iterations')
    
#     return v_grid_new, n_grid_new
    
# # Can check code against theoretical results by looking at spread and number of entrants
# # For spread of entrants, compare each firms entry to highest. Graph results
# # Keep in mind that spread_theory doesn't know that vacancies cannot be negative
# def p_spread(bl, v_grid, n_grid,
#            save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Baseline\Planner_Spread'):
    
#     gamma, k, eta, r, delta = bl.gamma, bl.k, bl.eta, bl.r, bl.delta
#     q, y_grid = bl.q, bl.y_grid
#     f = bl.f
        
#     theta = np.sum(v_grid) / (1 - np.sum(n_grid))
    
#     spread_prod = (c(v_grid[-1], gamma, k) - c(v_grid, gamma, k)) * (r + delta) / q(theta)
#     spread_theory = (y_grid[-1] - y_grid) 
#     spread_theory[v_grid == 0] = spread_prod[v_grid == 0]
#     dif = spread_theory - spread_prod
    
#     fig, ax = plt.subplots()
#     ax.plot(y_grid, spread_prod, lw=2, alpha=0.7, label='Production')
#     ax.plot(y_grid, spread_theory, lw=2, alpha=0.7, label='Theoretical')
#     ax.set(xlabel='$y$', ylabel='Spread')
#     ax.legend(loc='best')
    
    
#     if save:
#         fig.savefig(folder + file + '.pdf')
#     else:
#         plt.show()   
        
#     fig, ax = plt.subplots()
#     ax.plot(y_grid, dif, lw=2, alpha=0.7)
#     ax.set(xlabel='$y$', ylabel='Spread')
#     ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    
#     if save:
#         fig.savefig(folder + file + '_dif.pdf')
#     else:
#         plt.show()  
        
#     plt.close(fig='all')

# # For number of vacancies, just have two numbers to compare
# def p_entry(bl, v_grid, n_grid):
    
#     b, gamma, k, eta, r, delta = bl.b, bl.gamma, bl.k, bl.eta, bl.r, bl.delta
#     q, q_p, p, y_grid = bl.q, bl.q_p, bl.p, bl.y_grid
#     f = bl.f
    
#     theta = np.sum(v_grid) / (1 - np.sum(n_grid))
#     alpha = -(theta * q_p(theta)) / q(theta)
    
#     entry_theory = np.sum(v_grid * (y_grid - b))
#     entry_prod = (r + delta + alpha * theta * q(theta)) * np.sum(v_grid * c(v_grid, gamma, k)) / ((1 - alpha) * q(theta))
    
#     f.write(f'Entry from production gives {entry_prod:.4f}; Entry from theory gives {entry_theory:.4f} \n')
    
#     # While doing this, also check if u is consistent with LOM
#     u = 1 - np.sum(n_grid)
#     u_t = delta / (delta + bl.p(theta))    
#     f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
#     # Probably related to whether p and q are < 1. Check these
#     f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
# ##################################################################################################################

# # Outsourcer's Decentralized

# @vectorize
# def c_inv(c, gamma=2, k=1):
#     if c > 0:
#         return (c / gamma / k)**(1 / (gamma - 1))
#     else:
#         return 0

# # Do the same for outsourcers, who have C^H(v) = v**gamma_h
# @vectorize
# def C_h(v, gamma_h=1.5, k_h=1):
#     return k_h * v**gamma_h

# @vectorize
# def c_h(v, gamma_h=1.5, k_h=1):
#     if v > 0:
#         return k_h * gamma_h * v**(gamma_h - 1)
#     else:
#         return 0
     
# @vectorize
# def c_h_inv(c, gamma_h=1.5, k_h=1):
#     if c > 0:
#         return (c / gamma_h / k_h)**(1 / (gamma_h - 1))
#     else:
#         return 0
    
# # Set up the baseline environment, starting with a discrete model
# class OutsourcerDiscrete:
#     """
#     B is discount rate beta
#     r is interest rate defined by beta
#     b is home production
#     delta is job loss rate
#     delta_hat is firm destruction rate <= delta
#     eta is worker bargaining power    
#     phi is effectiveness of matching function
#     q is firm's matching function, worker's is p=theta*q(theta)
#     y_min, y_max, y_grid_size determine productivity grid
#     gamma is steepness of marginal cost
#     gamma_h is steepness of marginal cost for outsourcer
#     q is matching function of firms
#     p is matching function of workers
    
#     f is for printing
#     """
#     def __init__(self, B, b, delta, delta_hat, eta, phi, y_min, y_max, y_grid_size, gamma, gamma_h, k, k_h, f):
        
#         self.B, self.b, self.delta, self.eta, self.gamma, self.k = B, b, delta, eta, gamma, k
#         self.delta_hat, self.gamma_h, self.k_h = delta_hat, gamma_h, k_h 
#         self.r = 1 / B - 1
        
#         self.y_grid = np.linspace(y_min, y_max, y_grid_size)
#         self.y_grid_size = y_grid_size        
        
#         self.q = njit(lambda x: min(phi * x**(-1 / 2), 1))
#         self.p = njit(lambda x: min(phi * x**(1 / 2), 1))
#         self.q_p = njit(lambda x: -phi / 2 * x**(-3 / 2)) 
#         self.p_p = njit(lambda x: phi / 2 * x**(1 / 2))
        
#         self.f = f

        
# ###########################################
# # Try to find eqbm by chosing p_h and comparing QD to QS. Simialr method to one used above

# # First set up iteration
# def o_operator_factory(out):
    
#     B, r, b, delta, eta, gamma, k = out.B, out.r, out.b, out.delta, out.eta, out.gamma, out.k
#     delta_hat, gamma_h, k_h = out.delta_hat, out.gamma_h, out.k_h
#     y_grid, y_grid_size = out.y_grid, out.y_grid_size
#     y_grid_max = max(y_grid)
#     q, p = out.q, out.p
#     f = out.f
    
#     # Use v_grid, u, v_h, and y_hat to find pi, theta and f_grid (make sure v not 0)
#     @njit()
#     def get_dist(v_grid, v_h, u, y_hat):
#         v = max(np.sum(v_grid[y_grid < y_hat]), 1e-6)    
#         # Outsourcing firms not in f_grid
#         f_grid = np.minimum(v_grid / v, y_grid < y_hat)
#         theta = (v + v_h) / u
#         pi = v_h / (v + v_h)
        
#         return theta, f_grid, pi    
    
#     # Find wages for each postion
#     @njit()
#     def prices(v_grid, v_h, p_h, theta, f_grid, pi):
#         # Useful to define a value of search 
#         val_search = theta * ((1 - pi) * np.sum(c(v_grid, gamma, k) * f_grid) + pi * c_h(v_h, gamma_h, k_h))
# #         # For some reason, val_seach likes to sometimes return vectors, change this
# #         try:
# #             val_search = val_search[0]
# #         except:
# #             val_search = val_search
#         w_grid = b + eta * (y_grid - b + val_search)
#         omega = b + eta * (p_h - b + val_search)
                        
#         return w_grid, omega 
    
#     # Find y_hat using prices
#     @njit()
#     def get_y_hat(theta, w_grid, p_h):
#         J = (y_grid - w_grid) / (r + delta)
#         J_H = (y_grid - p_h) / (r + delta_hat)
#         choice = 1 * (J_H >= q(theta) * J) * (J_H > 0)
#         ind = np.searchsorted(choice, 0, side='right')
#         # If y_hat > y_max, return y_max + 1
#         if ind < y_grid_size:
#             y_hat = y_grid[ind]
#         else:
#             y_hat = y_grid_max + 1
#         return y_hat
    
#     # Find quantity supplied and quantity demanded for outsourcing
#     @njit()
#     def find_QS_QD(v_grid, v_h, u, y_hat):
#         # Find st st supply and demand of outsourcing
#         theta, f_grid, pi = get_dist(v_grid, v_h, u, y_hat)        
#         # Supply of outsourcing are matched outsourcers, not outsourcer vacancies
#         QS = v_h * q(theta) / delta
#         # Demand for outsourcing are matched firms, not firm vacancies
#         QD = np.sum(v_grid[y_grid >= y_hat]) / delta_hat
#         return QS, QD
    
#     @njit()
#     def iterate(v_grid, v_h, u, y_hat, p_h):
        
#         v_grid_new = np.empty(y_grid_size)
        
#         theta, f_grid, pi = get_dist(v_grid, v_h, u, y_hat)

#         w_grid, omega = prices(v_grid, v_h, p_h, theta, f_grid, pi)        
        
#         # Update y_hat
#         y_hat_new = get_y_hat(theta, w_grid, p_h)
        
#         # Should the relevant comparison be y_hat or y_hat_new? Make switching easier
#         y_comp = y_hat
        
#         # Firms who hire own workers (below y_comp)
#         v_grid_new[y_grid < y_comp] = (
#             c_inv(q(theta) * (y_grid[y_grid < y_comp] - w_grid[y_grid < y_comp]) / (r + delta), gamma, k))
        
#         # Firms who outsource workers (above y_comp)
#         v_grid_new[y_grid >= y_comp] = (
#             c_inv((y_grid[y_grid >= y_comp] - p_h) / (r + delta_hat), gamma, k))
        
#         # v_h_new chosen using free entry
#         v_h_new = c_h_inv(q(theta) * (p_h - omega) / (r + delta), gamma_h, k_h)
        
#         # Update u_new using employed workers
#         n_grid_new = q(theta) * v_grid_new[y_grid < y_comp] / delta
#         u_new = max(1 - np.sum(n_grid_new) - (q(theta) * v_h_new / delta), 1e-6)
        
# #         # If want to base on v_grid_new[y_grid >= y_comp] rather than v_h_new, use this
# #         m_grid_new = v_grid_new[y_grid >= y_comp] / delta_hat
# #         u_new = max(1 - np.sum(n_grid_new) - np.sum(n_grid_new), 1e-6)

#         return v_grid_new, v_h_new, u_new, y_hat_new
    
#     return iterate, get_dist, prices, find_QS_QD

# # Solve model given p_h (and other factors to speed up process)
# def solve_given_p(out, p_h, v_grid, v_h, u, y_hat, tol, max_iter, slow):
    
#     iterate, get_dist, prices, find_QS_QD = o_operator_factory(out)
    
#     # Set up initial loop parameters
#     i = 0
#     err = tol + 1    
    
#     # Update v_grid, v_h, u, and y_hat each period
#     while i < max_iter and err > tol:
#         v_grid_new, v_h_new, u_new, y_hat_new = iterate(v_grid, v_h, u, y_hat, p_h)
#         err_v = np.max(np.abs(v_grid_new - v_grid))
#         err_v_h = np.abs(v_h_new - v_h)
#         err_u = np.abs(u_new - u)
#         err_y_hat = np.abs(y_hat_new - y_hat)
#         err = max(err_v, err_v_h, err_u, err_y_hat)
#         i += 1
        
#         v_grid = slow * v_grid_new + (1 - slow) * v_grid
#         v_h = slow * v_h_new + (1 - slow) * v_h        
#         u = slow * u_new + (1 - slow) * u
#         y_hat = slow * y_hat_new + (1 - slow) * y_hat
        
#     return v_grid_new, v_h_new, u_new, y_hat_new

# # Find p_h by iteration
# def find_p(out, tol, max_iter, tol_i, max_iter_i, verbose=True, print_skip=100, slow_i=1, slow=1):
    
#     iterate, get_dist, prices, find_QS_QD = o_operator_factory(out)
    
#     # Set up initial guesses and loop parameters
#     p_h = np.mean(out.y_grid)
#     v_grid = np.ones(out.y_grid_size) / out.y_grid_size
#     u = 0.05
#     y_hat = p_h
#     v_h = 1
#     err_p = tol + 1
#     j = 0
#     s_t = 0
    
#     while j < max_iter:
#         v_grid, v_h, u, y_hat = solve_given_p(out, p_h, v_grid, v_h, u, y_hat, tol_i, max_iter_i, slow_i)
#         QS, QD = find_QS_QD(v_grid, v_h, u, y_hat)
#         err_p = QD - QS
        
#         j += 1
#         if verbose and j % print_skip == 0:
#             print(f'Error at iter {j} is {err_p}; price is {p_h}; QS is {QS}; QD is {QD}')
#         if verbose and j % print_skip == 1:
#             print(f'Error at iter {j} is {err_p}; price is {p_h}; QS is {QS}; QD is {QD}')
            
#         if np.abs(err_p) <= tol:
#             print(f'Converged in {j} iterations')
#             return v_grid, v_h, u, y_hat, p_h
#         else:
#             p_h = max(min(p_h + err_p * slow, np.max(out.y_grid)),0)
        
#     print(f'Failed to Converge. Error is {err_p}')    
#     return v_grid, v_h, u, y_hat, p_h

# # Can check code against theoretical results by looking at spread and number of entrants
# # For spread of entrants, 3 comparisons
# # 1. Above y_hat, compare to y_max
# # 2. Below y_hat, compare to y_max
# # 3. Below y_hat, compare to y_hat (or largest y below y_hat)
# # Graph results
# # Keep in mind that spread_theory doesn't know that vacancies cannot be negative
# def o_spread(out, v_grid, v_h, u, y_hat, p_h,
#            save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\Spread'):
    
#     gamma, k, eta, r, delta, delta_hat = out.gamma, out.k, out.eta, out.r, out.delta, out.delta_hat
#     gamma_h, k_h = out.gamma_h, out.k_h
#     q, y_grid = out.q, out.y_grid
#     f = out.f
    
#     iterate, get_dist, prices, find_QS_QD = o_operator_factory(out)
#     theta, f_grid, pi = get_dist(v_grid, v_h, u, y_hat)
    
#     ind = np.searchsorted(y_grid, y_hat)
    
#     spread_theory_1 = (c(v_grid[-1], gamma, k) - c(v_grid, gamma, k)) * (r + delta_hat)
    
#     spread_theory_2 = (c(v_grid[-1], gamma, k) * (r + delta_hat) 
#                        - (c(v_grid, gamma, k) - c_h(v_h, gamma_h, k_h)) * (r + delta) / ((1 - eta) * q(theta)))
    
#     spread_theory_3 = (c(v_grid[ind], gamma, k) - c(v_grid, gamma, k)) * (r + delta) / ((1 - eta) * q(theta)) 
        
#     spread_prod_y_max = y_grid[-1] - y_grid
#     spread_prod_y_max[v_grid == 0] = spread_theory_2[v_grid == 0]
#     spread_prod_y_hat = y_grid[ind] - y_grid
#     spread_prod_y_hat[v_grid == 0] = spread_theory_3[v_grid == 0]
    
#     dif_1 = spread_prod_y_max - spread_theory_1
#     dif_2 = spread_prod_y_max - spread_theory_2
#     dif_3 = spread_prod_y_hat - spread_theory_3
    
#     fig, ax = plt.subplots()
#     ax.plot(y_grid, spread_prod_y_max, lw=2, alpha=0.7, label='Productivity $y_{max}$')
#     ax.plot(y_grid[:ind], spread_prod_y_hat[:ind], lw=2, alpha=0.7, label=r'Productivity $\hat{y}$')
#     ax.plot(y_grid[ind:], spread_theory_1[ind:], lw=2, alpha=0.7, label='Theoretical 1')
#     ax.plot(y_grid[:ind], spread_theory_2[:ind], lw=2, alpha=0.7, label='Theoretical 2')
#     ax.plot(y_grid[:ind], spread_theory_3[:ind], lw=2, alpha=0.7, label='Theoretical 3')
#     ax.set(xlabel='$y$', ylabel='Spread')
#     ax.legend(loc='best')
    
    
#     if save:
#         fig.savefig(folder + file + '.pdf')
#     else:
#         plt.show()   
        
#     fig, ax = plt.subplots()
#     ax.plot(y_grid[ind:], dif_1[ind:], lw=2, alpha=0.7, label='Above')
#     ax.plot(y_grid[:ind], dif_2[:ind], lw=2, alpha=0.7, label='Above-Below')
#     ax.plot(y_grid[:ind], dif_3[:ind], lw=2, alpha=0.7, label='Below')
#     ax.set(xlabel='$y$', ylabel='Spread')
#     ax.legend(loc='best')
#     ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
    
#     if save:
#         fig.savefig(folder + file + '_dif.pdf')
#     else:
#         plt.show() 
        
#     plt.close(fig='all')
        

# # For number of vacancies, look at above and below y_hat
# def o_entry(out, v_grid, v_h, n_grid, n_h, y_hat, p_h):
    
#     gamma, k, eta, r, delta, delta_hat = out.gamma, out.k, out.eta, out.r, out.delta, out.delta_hat
#     gamma_h, k_h = out.gamma_h, out.k_h
#     q, p, y_grid = out.q, out.p, out.y_grid
#     f = out.f
    
#     u = 1 - np.sum(n_grid)
    
#     iterate, get_dist, prices, find_QS_QD = o_operator_factory(out)
#     theta, f_grid, pi = get_dist(v_grid, v_h, u, y_hat)
    
#     # Useful to define a value of search 
#     val_search = (eta / (1 - eta) * theta
#                   * ((1 - pi) * np.sum(c(v_grid, gamma, k) * f_grid) + pi * c_h(v_h, gamma_h, k_h)))
#     # For some reason, val_seach likes to sometimes return vectors, change this
#     try:
#         val_search = val_search[0]
#     except:
#         val_search = val_search
    
#     ind = np.searchsorted(y_grid, y_hat)
    
#     entry_theory_a = np.sum(v_grid[ind:] * (y_grid[ind:] - b))
#     entry_theory_b = np.sum(v_grid[:ind - 1] * (y_grid[:ind - 1] - b))
    
#     entry_code_a = np.sum(v_grid[ind:] * (val_search + (r + delta_hat) * c(v_grid[ind:], gamma, k)
#                                           + (r + delta) / ((1 - eta) * q(theta)) * c_h(v_h, gamma_h, k_h)))
#     entry_code_b = np.sum(v_grid[:ind] * (val_search
#                                           + (r + delta) / ((1 - eta) * q(theta)) * c(v_grid[:ind], gamma, k)))
    
#     f.write(f'Entry above from code gives {entry_code_a:.4f}; entry from theory gives {entry_theory_a:.4f} \n')
#     f.write(f'Entry below from code gives {entry_code_b:.4f}; entry from theory gives {entry_theory_b:.4f} \n')
    
#     # While doing this, also check if u is consistent with LOM
#     u_t = delta / (delta + p(theta))    
#     f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
#     # Probably related to whether p and q are < 1. Check these
#     f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
#     # What percent of firms outsource?
#     per_out = n_h / np.sum(n_grid) * 100
#     f.write(f'Percent of matches outsourcing is {per_out:2.0f} \n')
    
    
# #######################################################################################

# # Create graph of supply and demand for outsourced workers

# # First set up iteration
# def supply_demand_factory(out):
    
#     B, r, b, delta, eta, gamma, k = out.B, out.r, out.b, out.delta, out.eta, out.gamma, out.k
#     delta_hat, gamma_h, k_h = out.delta_hat, out.gamma_h, out.k_h
#     y_grid, y_grid_size = out.y_grid, out.y_grid_size
#     y_grid_max = max(y_grid)
#     q, p = out.q, out.p
#     f = out.f
    
#     # Use v_grid, u, v_h, and y_hat to find pi, theta and f_grid (make sure v not 0)
#     @njit()
#     def get_dist_SD(v_grid, v_h, u, y_hat):
#         v = max(np.sum(v_grid[y_grid < y_hat]), 1e-6)    
#         # Outsourcing firms not in f_grid
#         f_grid = np.minimum(v_grid / v, y_grid < y_hat)
#         theta = (v + v_h) / u
#         pi = v_h / (v + v_h)
        
#         return theta, f_grid, pi    
    
#     # Find wages for each postion
#     @njit()
#     def prices_SD(v_grid, v_h, p_h, theta, f_grid, pi):
#         # Useful to define a value of search 
#         val_search = theta * ((1 - pi) * np.sum(c(v_grid, gamma, k) * f_grid) + pi * c_h(v_h, gamma_h, k_h))
# #         # For some reason, val_seach likes to sometimes return vectors, change this
# #         try:
# #             val_search = val_search[0]
# #         except:
# #             val_search = val_search
#         w_grid = b + eta * (y_grid - b + val_search)
#         omega = b + eta * (p_h - b + val_search)
                        
#         return w_grid, omega 
    
#     # Find y_hat using prices
#     @njit()
#     def get_y_hat_SD(theta, w_grid, p_h):
#         J = (y_grid - w_grid) / (r + delta)
#         J_H = (y_grid - p_h) / (r + delta_hat)
#         choice = 1 * (J_H >= q(theta) * J) * (J_H > 0)
#         ind = np.searchsorted(choice, 0, side='right')
#         # If y_hat > y_max, return y_max + 1
#         if ind < y_grid_size:
#             y_hat = y_grid[ind]
#         else:
#             y_hat = y_grid_max + 1
#         return y_hat
    
#     @njit()
#     def iterate_SD(v_grid, v_h, u, y_hat, p_h):
        
#         v_grid_new = np.empty(y_grid_size)
        
#         theta, f_grid, pi = get_dist_SD(v_grid, v_h, u, y_hat)

#         w_grid, omega = prices_SD(v_grid, v_h, p_h, theta, f_grid, pi)        
        
#         y_hat_new = get_y_hat_SD(theta, w_grid, p_h)
        
#         # Should the relevant comparison be y_hat or y_hat_new? Make switching easier
#         y_comp = y_hat
        
#         # Firms who hire own workers (below y_comp)
#         v_grid_new[y_grid < y_comp] = (
#             c_inv(q(theta) * (y_grid[y_grid < y_comp] - w_grid[y_grid < y_comp]) / (r + delta), gamma, k))
        
#         # Firms who outsource workers (above y_comp)
#         v_grid_new[y_grid >= y_comp] = (
#             c_inv((y_grid[y_grid >= y_comp] - p_h) / (r + delta_hat), gamma, k))
        
#         # v_h_new chosen using free entry
#         v_h_new = c_h_inv(q(theta) * (p_h - omega) / (r + delta), gamma_h, k_h)
        
#         # Update u_new using unemployed workers
#         n_grid_new = q(theta) * v_grid_new[y_grid < y_comp] / delta
#         u_new = max(1 - np.sum(n_grid_new) - (q(theta) * v_h_new / delta), 1e-6)
        
# #         # If want to base on v_grid_new[y_grid >= y_comp] rather than v_h_new, use this
# #         m_grid_new = v_grid_new[y_grid >= y_comp] / delta_hat
# #         u_new = max(1 - np.sum(n_grid_new) - np.sum(n_grid_new), 1e-6)
        
#         return v_grid_new, v_h_new, u_new, y_hat_new
    
#     return iterate_SD, get_dist_SD, prices_SD

# # Solve model given p_h
# def solve_model_SD(out, p_h, tol, max_iter, slow):
    
#     iterate_SD, get_dist_SD, prices_SD = supply_demand_factory(out)
#     f = out.f
    
#     # Set up initial guesses and loop parameters
#     v_grid = np.ones(out.y_grid_size) / out.y_grid_size
#     u = 0.05
#     y_hat = p_h
#     v_h = 1
#     i = 0
#     err = tol + 1    
    
#     # Update v_grid, v_h, u, and y_hat each period
#     while i < max_iter and err > tol:
#         v_grid_new, v_h_new, u_new, y_hat_new = iterate_SD(v_grid, v_h, u, y_hat, p_h)
#         err_v = np.max(np.abs(v_grid_new - v_grid))
#         err_v_h = np.abs(v_h_new - v_h)
#         err_u = np.abs(u_new - u)
#         err_y_hat = np.abs(y_hat_new - y_hat)
#         err = max(err_v, err_v_h, err_u, err_y_hat)
#         i += 1
        
#         v_grid = slow * v_grid_new + (1 - slow) * v_grid
#         v_h = slow * v_h_new + (1 - slow) * v_h        
#         u = slow * u_new + (1 - slow) * u
#         y_hat = slow * y_hat_new + (1 - slow) * y_hat
        
#     return v_grid_new, v_h_new, u_new, y_hat_new, i, err


# def solve_QS_QD(out, p_h_min, p_h_max, p_h_grid_size, tol, max_iter, slow):
    
#     p_h_grid = np.linspace(p_h_min, p_h_max, p_h_grid_size, endpoint=False)
#     f = out.f
    
#     iterate_SD, get_dist_SD, prices_SD = supply_demand_factory(out)
    
#     # Create grid for QS and QD
#     QS_grid, QD_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
#     # Create a grid for theta, v_h, and va_grid (vacancies above y_hat)
#     theta_grid, v_h_grid, va_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size), np.empty(p_h_grid_size)
#     # Create grid for total firm vacancies
#     tot_firm_grid = np.empty(p_h_grid_size)
#     # Create grid for y_hat and omega
#     y_hat_grid, omega_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
#     # Create a grid for u
#     u_grid = np.empty(p_h_grid_size)
#     # Create grid for convergence and err
#     converge_grid, err_grid = np.empty(p_h_grid_size), np.empty(p_h_grid_size)
#     # List the maximum price with positive demand, start with p_h_max
#     i_max = p_h_max
    
#     # For each p_h, find QS and QD, which requires finding theta too
#     for i, p_h in enumerate(p_h_grid):
        
#         v_grid, v_h, u, y_hat, loops, err = solve_model_SD(out, p_h, tol, max_iter, slow)
#         theta, f_grid, pi = get_dist_SD(v_grid, v_h, u, y_hat)
#         # Supply of outsourcing are matched outsourcers, not outsourcer vacancies
#         QS_grid[i] = v_h * out.q(theta) / out.delta
#         v_h_grid[i] = v_h
#         # Demand for outsourcing are matched firms, not firm vacancies
#         QD_grid[i] = np.sum(v_grid[out.y_grid >= y_hat]) / out.delta_hat
#         va_grid[i] = np.sum(v_grid[out.y_grid >= y_hat])
#         # Total firm vacancies
#         tot_firm_grid[i] = QD_grid[i] + np.sum(v_grid[out.y_grid < y_hat]) * out.q(theta) / out.delta
#         y_hat_grid[i] = y_hat
#         theta_grid[i] = theta
#         _, omega = prices_SD(v_grid, v_h, p_h, theta, f_grid, pi)
#         omega_grid[i] = omega
#         u_grid[i] = u
#         converge_grid[i] = (loops < max_iter)
#         err_grid[i] = err
# #         f.write(f'Cost is {out.q(theta) * (p_h - omega) / (r + delta)}')
        
#         # If QD_grid[i] == 0, record i as i_max and break loop
#         if QD_grid[i] == 0:
#             i_max = i
#             break
        
#     return (QS_grid, QD_grid, tot_firm_grid, v_h_grid, va_grid, theta_grid,
#             y_hat_grid, omega_grid, u_grid, converge_grid, err_grid, i_max) 

# # Graph results
# def supply_demand(out, p_h_min, p_h_max, p_h_grid_size, tol=1e-4, max_iter=1000, slow=1/100,
#            save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\SD'):
    
#     p_h_grid = np.linspace(p_h_min, p_h_max, p_h_grid_size, endpoint=False)
    
#     (QS_grid, QD_grid, tot_firm_grid, v_h_grid, va_grid, theta_grid,
#      y_hat_grid, omega_grid, u_grid, converge_grid, err_grid, i_max
#     ) = solve_QS_QD(out, p_h_min, p_h_max, p_h_grid_size, tol, max_iter, slow)
        
#     fig, ax = plt.subplots()
#     ax.plot(QS_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Supply')
#     ax.plot(QD_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Demand')
#     ax.set(xlabel='$n$', ylabel='$p_h$')
#     ax.legend(loc='best')
    
    
#     if save:
#         fig.savefig(folder + file + '.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(QS_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Supply')
#     ax.plot(v_h_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Vacancies')
#     ax.set(xlabel='$n$', ylabel='$p_h$')
#     ax.legend(loc='best')
    
    
#     if save:
#         fig.savefig(folder + file + '_Supply.pdf')
#     else:
#         plt.show()
    
        
#     fig, ax = plt.subplots()
#     ax.plot(QD_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Demand')
#     ax.plot(va_grid[:i_max], p_h_grid[:i_max], lw=2, alpha=0.7, label='Vacancies')
#     ax.set(xlabel='$n$', ylabel='$p_h$')
#     ax.legend(loc='best')
    
    
#     if save:
#         fig.savefig(folder + file + '_Demand.pdf')
#     else:
#         plt.show()
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], QD_grid[:i_max] - QS_grid[:i_max], lw=2, alpha=0.7)
#     ax.axhline(y=0, ls=':', c='k')
#     ax.set(xlabel='$p_h$', ylabel=r'$QD - QS$')
    
    
#     if save:
#         fig.savefig(folder + file + '_qd_min_qs.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], QD_grid[:i_max] / tot_firm_grid[:i_max] * 100, lw=2, alpha=0.7)
#     ax.axhline(y=0, ls=':', c='k')
#     ax.set(xlabel='$p_h$', ylabel=r'%')
    
    
#     if save:
#         fig.savefig(folder + file + '_per_out.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], theta_grid[:i_max], lw=2, alpha=0.7)
#     ax.set(xlabel='$p_h$', ylabel=r'$\theta$')
    
    
#     if save:
#         fig.savefig(folder + file + '_theta.pdf')
#     else:
#         plt.show() 
    
        
#     p_grid = np.empty_like(theta_grid)
#     q_grid = np.empty_like(theta_grid)
#     for ind, theta in enumerate(theta_grid):
#         p_grid[ind] = out.p(theta)
#         q_grid[ind] = out.q(theta)
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], p_grid[:i_max], lw=2, alpha=0.7, label=r'$p(\theta)$')
#     ax.plot(p_h_grid[:i_max], q_grid[:i_max], lw=2, alpha=0.7, label=r'$q(\theta)$')
#     ax.set(xlabel='$p_h$', ylabel=r'Matching Probability')
#     ax.legend()
    
    
#     if save:
#         fig.savefig(folder + file + '_match_prob.pdf')
#     else:
#         plt.show() 
    
        
        
#     y_min = min(out.y_grid)
#     y_max = max(out.y_grid)
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], y_hat_grid[:i_max], lw=2, alpha=0.7)
#     ax.axhline(y=y_min, ls=':', c='k')
#     ax.axhline(y=y_max, ls=':', c='k')
#     ax.set(xlabel='$p_h$', ylabel='$y^*$')
    
    
#     if save:
#         fig.savefig(folder + file + '_y_hat.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], omega_grid[:i_max], lw=2, alpha=0.7)
#     ax.set(xlabel='$p_h$', ylabel='$\omega$')
    
    
#     if save:
#         fig.savefig(folder + file + '_omega.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], u_grid[:i_max], lw=2, alpha=0.7, label='u_model')    
#     ax.set(xlabel='$p_h$', ylabel='$u$')
    
    
#     if save:
#         fig.savefig(folder + file + '_u.pdf')
#     else:
#         plt.show() 
        
    
#     u_lom_grid = np.empty_like(theta_grid)
#     for ind, theta in enumerate(theta_grid):
#         u_lom_grid[ind] = delta / (delta + out.p(theta))
    
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], u_grid[:i_max], lw=2, alpha=0.7, label='u_model')    
#     ax.plot(p_h_grid[:i_max], u_lom_grid[:i_max], lw=2, alpha=0.7, label='u_lom')
#     ax.set(xlabel='$p_h$', ylabel='$u$', title='Unemployment Given Price $p_h$')
#     ax.legend()
    
    
#     if save:
#         fig.savefig(folder + file + '_u_check.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], converge_grid[:i_max], lw=2, alpha=0.7)
#     ax.set(xlabel='$p_h$', ylabel='converge', title='Converge Given Price $p_h$')
#     ax.set_ylim(-0.05, 1.05)
    
    
#     if save:
#         fig.savefig(folder + file + '_converge.pdf')
#     else:
#         plt.show() 
    
        
#     fig, ax = plt.subplots()
#     ax.plot(p_h_grid[:i_max], err_grid[:i_max], lw=2, alpha=0.7)
#     ax.set(xlabel='$p_h$', ylabel='err', title='Err Given Price $p_h$')
    
    
#     if save:
#         fig.savefig(folder + file + '_err.pdf')
#     else:
#         plt.show()
        
#     plt.close(fig='all')
    
        
# #################################################################################################


# # Outsourcer Planning

# def o_p_operator_factory(out):
#     "Use a baseline case out and create a function to update Lagrange multipliers and solve planner's problem"
    
#     B, r, b, delta, eta, gamma, k = out.B, out.r, out.b, out.delta, out.eta, out.gamma, out.k
#     delta_hat, gamma_h, k_h = out.delta_hat, out.gamma_h, out.k_h
#     y_grid, y_grid_size = out.y_grid, out.y_grid_size
#     y_grid_max = max(y_grid)
#     q, q_p, p = out.q, out.q_p, out.p
#     f = out.f
    
#     # Use y_hat to find it's closest value on y_grid
#     @njit()
#     def find_y_hat(y_hat):
#         ind = np.searchsorted(y_grid, y_hat, side='left')
#         # If always above, set equal to top
#         ind = np.int64(min(ind, y_grid_size))
#         return y_grid[ind]    

#     # Use v_grid, v_h, n_grid, n_h, and y_hat to find u and theta 
#     @njit()
#     def o_p_get_dist(v_grid, v_h, n_grid, n_h, y_hat):
#         u = max(1 - np.sum(n_grid[y_grid < y_hat]) - n_h, 1e-6)
#         theta = max(np.sum(v_grid[y_grid < y_hat]) + v_h, 1e-6) / u
#         return u, theta
    
#     # lam is lagrangian for hiring firms
#     # mu is lagrangian for outsourcing firms
#     # rho is lagrangian for outsourcer
#     # psi is lagrangian for outsourcing firms less than outsourcer
#     # Find all lagrangians through iteration 
#     @njit()
#     def find_lagrangians(v_grid, v_h, y_hat, u, theta, lam, mu, rho, psi, tol_l=1e-4, max_iter_l=1000, slow_l=1e-4):       
#         i = 0
#         err = tol_l + 1
        
# #         lam = np.ones(y_grid_size)
# #         mu = np.ones(y_grid_size)
# #         rho = np.float32(1)
# #         psi = np.float32(1)

# #         # Another way to calculate psi
# #         psi = (y_grid[-1] + (r + delta_hat) * c(v_grid[-1], gamma, k)) / (1 + r)

#         while i < max_iter_l and err > tol_l:
            
#             y_hat = find_y_hat(y_hat)
            
#             v_sum = np.sum(v_grid[y_grid < y_hat] * lam[y_grid < y_hat]) + v_h * rho
#             lam_new = (y_grid - b + (theta * q_p(theta) * v_sum / u)) / (r + delta)
#             mu_new = (y_grid - (1 + r) * psi) / (r + delta_hat)
#             rho_new = (-b + (1 + r) * psi + (theta * q_p(theta) * v_sum / u)) / (r + delta)
            
#             err_lam = np.max(np.abs(lam_new - lam))
#             err_mu = np.max(np.abs(mu_new - mu))
#             err_rho = np.abs(rho_new - rho)
#             err = max(err_lam, err_mu, err_rho)
            
#             i += 1
            
#             lam = slow_l * lam_new + (1 - slow_l) * lam
#             mu = slow_l * mu_new + (1 - slow_l) * mu
#             rho = slow_l * rho_new + (1 - slow_l) * rho
            
#         if i == max_iter_l:
#             print('lagrangians failed to converge')
        
#         return lam_new, mu_new, rho_new
    
#     # update v_grid, v_h, n_grid, n_h, and y_hat
#     @njit()
#     def update_grids(v_grid, v_h, n_grid, n_h, y_hat, lam, mu, rho, psi, tol_l, max_iter_l, slow=1, slow_l=1e-4):
        
#         y_hat = find_y_hat(y_hat)
        
#         u, theta = o_p_get_dist(v_grid, v_h, n_grid, n_h, y_hat)

#         lam_new, mu_new, rho_new = find_lagrangians(
#             v_grid, v_h, y_hat, u, theta, lam, mu, rho, psi, tol_l, max_iter_l, slow_l
#         )
        
#         v_grid_new, n_grid_new = np.empty(y_grid_size), np.empty(y_grid_size)
        
#         # Given a lagrangian, repeatedly update v_grid and n_grid hopefully will be faster
#         for j in range(1 / slow):
            
#             v_sum = np.sum(v_grid[y_grid < y_hat] * lam_new[y_grid < y_hat]) + rho * v_h
            
#             cost = lam_new[y_grid < y_hat] * q(theta) + (q_p(theta) / u) * v_sum           
#             v_grid_new[y_grid < y_hat] = c_inv(cost, gamma, k)

#             v_grid_new[y_grid >= y_hat] = c_inv(mu_new[y_grid >= y_hat], gamma, k)

#             cost_h = rho_new * q(theta) + (q_p(theta) / u) * v_sum
#             v_h_new = c_h_inv(cost_h, gamma_h, k_h)

#             n_grid_new[y_grid < y_hat] = q(theta) * v_grid_new[y_grid < y_hat] / delta

#             n_grid_new[y_grid >= y_hat] = v_grid_new[y_grid >= y_hat] / delta_hat

#             n_h_new = q(theta) * v_h_new / delta
            
#             comp = lam_new * q(theta) + (q_p(theta) / u) * v_sum- mu_new
#             ind_new = np.argmin(np.abs(comp))
#             y_hat_new = y_grid[ind_new]
            
#             v_grid = v_grid_new * slow + v_grid * (1 - slow) 
#             v_h = v_h_new * slow + v_h * (1 - slow)
#             n_grid = n_grid_new * slow + n_grid * (1 - slow)
#             n_h = n_h_new * slow + n_h * (1 - slow)
#             y_hat = y_hat_new * slow + y_hat * (1 - slow)
            
#             y_hat = find_y_hat(y_hat)        
#             u, theta = o_p_get_dist(v_grid, v_h, n_grid, n_h, y_hat)
        
#         return v_grid_new, v_h_new, n_grid_new, n_h_new, y_hat_new, lam_new, mu_new, rho_new
    
#     return update_grids, find_y_hat, o_p_get_dist, find_lagrangians

# # Solve model
# def solve_o_p_model(out, guess, tol_l=1e-4, tol=1e-4, max_iter_l=1e3, max_iter=1e4, verbose=True, print_skip=1e3,
#                   slow_i=1, slow_l=1e-2, slow=1):   

#     update_grids, _, _, _ = o_p_operator_factory(out)
    
#     y_grid, r, b, delta, delta_hat = out.y_grid, out.r, out.b, out.delta, out.delta_hat
#     gamma, gamma_h, k, k_h = out.gamma, out.gamma_h, out.k, out.k_h 
#     q = out.q
#     f = out.f
    
#     # Set up initial guesses and loop parameters

#     v_grid, v_h, u, y_hat = guess
    
#     theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
#     n_grid = (q(theta) * v_grid) / delta
#     n_grid[y_grid > y_hat] = v_grid[y_grid > y_hat] / delta_hat
#     n_h = (q(theta) * v_h) / delta
    
#     # Carry around lagrangian to make re-estimating faster
#     lam = (y_grid - b) / (r + delta)
#     psi = (y_grid[-1] - (r + delta_hat) * c(v_grid[-1], gamma, k)) / (1 + r)
#     mu = (y_grid - (1 + r) * psi) / (r + delta_hat)
#     rho = (-b + (1 + r) * psi) / (r + delta)
    
#     i = 0
#     err = tol + 1 
    
#     # Update v_grid, v_h, n_grid, n_h, and lagrangian each period
#     while i < max_iter and err > tol:
#         v_grid_new, v_h_new, n_grid_new, n_h_new, y_hat_new, lam_new, mu_new, rho_new = update_grids(
#             v_grid, v_h, n_grid, n_h, y_hat, lam, mu, rho, psi, tol_l, max_iter_l, slow_i, slow_l)
        
#         o_demand = np.sum(n_grid_new[out.y_grid > y_hat_new])
#         o_supply = n_h_new
#         err_o = o_demand - o_supply
#         # Change psi based on error. If o_supply is 0, add 1 to get started
#         psi += (err_o * slow) * (o_supply > 0) + 1 * (o_supply <= 0)
#         err_v = np.max(np.abs(v_grid_new - v_grid))
#         err_n = np.max(np.abs(n_grid_new - n_grid))
#         err_v_h = np.abs(v_h_new - v_h)
#         err_n_h = np.abs(n_h_new - n_h)
#         err_y_hat = np.abs(y_hat_new - y_hat)
#         err = max(err_v, err_n, err_v_h, err_n_h, err_y_hat, np.abs(err_o))
#         i += 1
        
#         if verbose and i % print_skip == 0:
#             print(f'Error at iter {i} is {err}; QS is {o_supply}, QD is {o_demand}')
#             print(y_hat_new, v_grid_new[-1])
            
#         if verbose and i % print_skip == 1:
#             print(f'Error at iter {i} is {err}; QS is {o_supply}, QD is {o_demand}')
#             print(y_hat_new, v_grid_new[-1])
        
# #         if verbose and i >= 0 and i <= 5:
# #             print(f'Error at iter {i} is {err}')
# #             print(y_hat_new, v_grid_new[-1])
        
#         v_grid = v_grid_new * slow + v_grid * (1 - slow)
#         v_h = v_h_new * slow + v_h * (1 - slow)
#         n_grid = n_grid_new * slow + n_grid * (1 - slow)
#         n_h = n_h_new * slow + n_h * (1 - slow)
#         y_hat = y_hat_new * slow + y_hat * (1 - slow)
#         lam = lam_new * slow + lam * (1 - slow)
#         mu = mu_new * slow + mu * (1 - slow)
#         rho = rho_new * slow + rho * (1 - slow)
        
        
#     if i == max_iter:
#         print("Failed to Converge")
        
#     if verbose and i < max_iter:
#         print(f'Converged in {i} iterations')
    
#     return v_grid_new, v_h_new, n_grid_new, n_h_new, y_hat_new 

# # Can check code against theoretical results by looking at spread and number of entrants
# # For spread of entrants, 3 comparisons
# # 1. Above y_hat, compare to y_max
# # 2. Below y_hat, compare to y_max
# # 3. Below y_hat, compare to y_hat (or largest y below y_hat)
# # Graph results
# # Keep in mind that spread_theory doesn't know that vacancies cannot be negative
# def o_p_spread(out, v_grid, v_h, n_grid, n_h, y_hat,
#              save=False, folder=r'C:\Users\spspi\Dropbox\Documents\Outsourcing\Figures', file='\Outsourcers\Planner_Spread'):
    
#     gamma, k, eta, r, delta, delta_hat = out.gamma, out.k, out.eta, out.r, out.delta, out.delta_hat
#     gamma_h, k_h = out.gamma_h, out.k_h
#     q, y_grid = out.q, out.y_grid
#     f = out.f
     
#     theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / (1 - np.sum(n_grid[y_grid < y_hat]) - n_h)
    
#     ind = np.searchsorted(y_grid, y_hat)
    
#     spread_theory_1 = (c(v_grid[-1], gamma, k) - c(v_grid, gamma, k)) * (r + delta_hat)
    
#     spread_theory_2 = (c(v_grid[-1], gamma, k) * (r + delta_hat) 
#                        - (c(v_grid, gamma, k) - c_h(v_h, gamma_h, k_h)) * (r + delta) / q(theta))
    
#     spread_theory_3 = (c(v_grid[ind - 1], gamma, k) - c(v_grid, gamma, k)) * (r + delta) / q(theta)
        
#     spread_prod_y_max = y_grid[-1] - y_grid
#     spread_prod_y_max[v_grid == 0] = spread_theory_2[v_grid == 0]
#     spread_prod_y_hat = y_grid[ind] - y_grid
#     spread_prod_y_hat[v_grid == 0] = spread_theory_3[v_grid == 0]
    
#     dif_1 = spread_prod_y_max - spread_theory_1
#     dif_2 = spread_prod_y_max - spread_theory_2
#     dif_3 = spread_prod_y_hat - spread_theory_3
    
#     fig, ax = plt.subplots()
#     ax.plot(y_grid, spread_prod_y_max, lw=2, alpha=0.7, label='Productivity $y_{max}$')
#     ax.plot(y_grid[:ind], spread_prod_y_hat[:ind], lw=2, alpha=0.7, label=r'Productivity $\hat{y}$')
#     ax.plot(y_grid[ind:], spread_theory_1[ind:], lw=2, alpha=0.7, label='Theoretical 1')
#     ax.plot(y_grid[:ind], spread_theory_2[:ind], lw=2, alpha=0.7, label='Theoretical 2')
#     ax.plot(y_grid[:ind], spread_theory_3[:ind], lw=2, alpha=0.7, label='Theoretical 3')
#     ax.set(xlabel='$y$', ylabel='Spread', title='Spread of Vacancies')
#     ax.legend(loc='best')
    
#     if save:
#         fig.savefig(folder + file + '.pdf')
#     else:
#         plt.show()   
        
#     fig, ax = plt.subplots()
#     ax.plot(y_grid[ind:], dif_1[ind:], lw=2, alpha=0.7, label='Above')
#     ax.plot(y_grid[:ind], dif_2[:ind], lw=2, alpha=0.7, label='Above-Below')
#     ax.plot(y_grid[:ind], dif_3[:ind], lw=2, alpha=0.7, label='Below')
#     ax.set(xlabel='$y$', ylabel='Spread', title='Difference in Spread of Vacancies')
#     ax.legend(loc='best')
#     ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
    
#     if save:
#         fig.savefig(folder + file + '_dif.pdf')
#     else:
#         plt.show() 
        
# # For number of vacancies, look at above and below y_hat
# def o_p_entry(out, v_grid, v_h, n_grid, n_h, y_hat):
    
#     b, gamma, k, eta, r, delta, delta_hat = out.b, out.gamma, out.k, out.eta, out.r, out.delta, out.delta_hat
#     gamma_h, k_h = out.gamma_h, out.k_h
#     q, q_p, p, y_grid = out.q, out.q_p, out.p, out.y_grid
#     f = out.f
     
#     u = 1 - np.sum(n_grid[y_grid < y_hat]) - n_h    
#     theta = (np.sum(v_grid[y_grid < y_hat]) + v_h) / u
#     alpha = - theta * q_p(theta) / q(theta)
    
#     ind = np.searchsorted(y_grid, y_hat)
    
#     entry_theory_a = np.sum(v_grid[ind:] * (y_grid[ind:] - b))
#     entry_theory_b = np.sum(v_grid[:ind] * (y_grid[:ind] - b))
    
#     # Find psi to estimate correct entry
#     psi = (y_grid[-1] - (r + delta_hat) * c(v_grid[-1], gamma, k)) / (1 + r)
    
#     psi_min_b = (1 + r) * psi - b
#     entry_code_a = ((r + delta_hat) * np.sum(v_grid[ind:] * c(v_grid[ind:], gamma, k))
#                     + psi_min_b * np.sum(v_grid[ind:]))
#     entry_code_b = (((r + delta + theta * alpha * q(theta)) / ((1 - alpha) * q(theta))) 
#                     * (np.sum(v_grid[:ind] * c(v_grid[:ind], gamma, k)) + c_h(v_h, gamma_h, k_h) * v_h)
#                     - psi_min_b * v_h)
    
#     f.write(f'Entry above from code gives {entry_code_a:.4f}; entry from theory gives {entry_theory_a:.4f} \n')
#     f.write(f'Entry below from code gives {entry_code_b:.4f}; entry from theory gives {entry_theory_b:.4f} \n')
    
#     # While doing this, also check if u is consistent with LOM
#     u_t = delta / (delta + p(theta))  
#     f.write(f'unemployment in model is {u:.4f}; according to LOM, should be {u_t:.4f} \n')
#     # Probably related to whether p and q are < 1. Check these
#     f.write(f'Employees match w/prob {p(theta):.2f}, employers match w/prob {q(theta):.2f} \n')
    
#     # Check if outsourcing firms equals outsourcer firms
#     f.write(f'Number of outsourcing matches is {np.sum(n_grid[ind:]):.5f}, number of outsourcer matches is {n_h:.5f} \n')
    
#     # What percent of firms outsource?
#     per_out = n_h / np.sum(n_grid) * 100
#     f.write(f'Percent of matches outsourcing is {per_out:2.0f} \n')
    
# ########################################################################################################
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# # Run code

# B = 0.95
# r = 1 / B - 1
# delta = 0.03
# eta = 0.5
# phi = 6e-1 
# y_min = 8
# y_max = 10
# b = y_min * .4
# y_grid_size = 4000
# gamma = 2
# delta_hat = 0.01
# gamma_h = 2
# k = 7e5
# k_h = 1e2


# f.write(f"""In this file, we have parameters: \n
# B = {B} \n
# r = {r} \n
# delta = {delta} \n
# delta_hat = {delta_hat} \n 
# eta = {eta} \n
# phi = {phi} \n 
# y_min = {y_min} \n 
# y_max = {y_max} \n
# y_grid_size = {y_grid_size} \n
# b = {b} \n 
# gamma = {gamma} \n 
# gamma_h = {gamma_h} \n 
# k = {k} \n 
# k_h = {k_h} \n""")

# bl = BaselineDiscrete(B, b, delta, eta, phi, y_min, y_max, y_grid_size, gamma, k, f)
# out = OutsourcerDiscrete(B, b, delta, delta_hat, eta, phi, y_min, y_max, y_grid_size, gamma, gamma_h, k, k_h, f)

# ###########################################
# f.write(" \n For Baseline; Decentralized Case \n")

# tol_1 = 1e-8
# max_iter_1 = 1e6
# slow_1 = 1e-3
# print_skip = 1e3
# v_grid_b, u_b = solve_model(bl, tol=tol_1, verbose=True, print_skip=print_skip, max_iter=max_iter_1, slow=slow_1)

# f.write(f"""
# tol_1 = {tol_1} \n
# max_iter_1 = {max_iter_1} \n
# slow_1 = {slow_1} \n""")

# iterate, get_dist, wages = operator_factory(bl)
# theta_b, f_grid_b = get_dist(v_grid_b, u_b)
# w_grid_b = wages(v_grid_b, theta_b, f_grid_b)

# n_grid_b = bl.q(theta_b) * v_grid_b / delta

# b_spread(bl, v_grid_b, u_b, save=True, folder=folder, file=r'\baseline_spread')
# b_entry(bl, v_grid_b, u_b)

# #######################################
# f.write(" \n For Baseline; Planner's Case \n")

# guess = [v_grid_b, u_b]
# tol_2 = 1e-8
# max_iter_2 = 1e6
# tol_l_2 = 1e-8
# max_iter_l_2 = 1e6
# ps_2 = 2e2
# slow_2 = 5e-3
# slow_l_2 = 1e-2 

# f.write(f"""
# tol_2 = {tol_2} \n
# max_iter_2 = {max_iter_2} \n
# tol_l_2 = {tol_l_2} \n
# max_iter_l_2 = {max_iter_l_2} \n
# slow_2 = {slow_2} \n
# slow_l_2 = {slow_l_2} \n""")

# v_grid_p, n_grid_p = solve_p_model(
#     bl, guess, tol_l=tol_l_2, tol=tol_2, max_iter_l=max_iter_l_2,
#     max_iter=max_iter_2, verbose=True, print_skip=ps_2, slow=slow_2, slow_l=slow_l_2)

# u_p = 1 - np.sum(n_grid_p)
# p_spread(bl, v_grid_p, u_p, save=True, folder=folder, file=r'\baseline_p_spread')
# p_entry(bl, v_grid_p, u_p)

# ###########################################
# f.write(" \n For Outsourcers; Decentralized Case \n")

# tol_3 = 1e-8
# max_iter_3 = 5e2
# tol_i_3 = 1e-8
# max_iter_i_3 = 1e4
# print_skip_3 = 5e1
# slow_i_3 = 1e-2
# slow_3 = 1e-1

# f.write(f"""
# tol_3 = {tol_3} \n
# max_iter_3 = {max_iter_3} \n
# tol_i_3 = {tol_i_3} \n
# max_iter_i_3 = {max_iter_i_3} \n
# slow_i_3 = {slow_i_3} \n
# slow_3 = {slow_3} \n""")

# v_grid_o, v_h_o, u_o, y_hat_o, p_h_o = find_p(
#     out, tol_3, max_iter_3, tol_i_3, max_iter_i_3, verbose=True, print_skip=print_skip_3, slow_i=slow_i_3, slow=slow_3)

# iterate, get_dist, prices, find_QS_QD = o_operator_factory(out)

# theta_o, f_grid_o, pi_o = get_dist(v_grid_o, v_h_o, u_o, y_hat_o)
# w_grid_o, omega_o = prices(v_grid_o, v_h_o, p_h_o, theta_o, f_grid_o, pi_o)

# o_spread(out, v_grid_o, v_h_o, u_o, y_hat_o, p_h_o,
#            save=True, folder=folder, file=r'\outsourcer_spread')

# n_grid_o = np.empty(y_grid_size)

# n_grid_o[out.y_grid < y_hat_o] = out.q(theta_o) * v_grid_o[out.y_grid < y_hat_o] / delta
# n_grid_o[out.y_grid >= y_hat_o] = v_grid_o[out.y_grid >= y_hat_o] / delta_hat

# n_h_o = out.q(theta_o) * v_h_o / delta

# o_entry(out, v_grid_o, v_h_o, n_grid_o, n_h_o, y_hat_o, p_h_o)



# ###########################################
# f.write(" \n For Outsourcers; Planner's Case \n")

# tol_4 = 1e-8
# max_iter_4 = 5e2
# tol_l_4 = 1e-8
# max_iter_l_4 = 5e5
# ps_4 = 5e1
# slow_i_4 = 1e-4
# slow_l_4 = 1e-2
# slow_4 = 1e-1

# f.write(f"""
# tol_4 = {tol_4} \n
# max_iter_4 = {max_iter_4} \n
# tol_l_4 = {tol_l_4} \n
# max_iter_l_4 = {max_iter_l_4} \n
# slow_i_4 = {slow_i_4} \n
# slow_l_4 = {slow_l_4} \n
# slow_4 = {slow_4} \n""")

# guess_o = [v_grid_o, v_h_o, u_o, y_hat_o]

# v_grid_o_p, v_h_o_p, n_grid_o_p, n_h_o_p, y_hat_o_p = solve_o_p_model(
#     out, guess=guess_o, tol_l=tol_l_4, tol=tol_4, max_iter_l=max_iter_l_4, max_iter=max_iter_4,
#     verbose=True, print_skip=ps_4, slow_i=slow_i_4, slow_l=slow_l_4, slow=slow_4)

# u_o_p = 1 - np.sum(n_grid_o_p[out.y_grid < y_hat_o_p]) - n_h_o_p

# o_p_spread(out, v_grid_o_p, v_h_o_p, n_grid_o_p, n_h_o_p, y_hat_o_p,
#            save=True, folder=folder, file=r'\outsourcer_p_spread')

# o_p_entry(out, v_grid_o_p, v_h_o_p, n_grid_o_p, n_h_o_p, y_hat_o_p)

# ###########################################
# f.write(" \n For Outsourcers; Supply and Demand \n")

# y_grid = out.y_grid

# p_h_min = min(y_grid)
# p_h_max = max(y_grid)
# p_h_grid_size = 100
# tol_5 = 1e-8
# max_iter_5 = 1e5
# slow_5 = 1e-2

# f.write(f"""
# tol_5 = {tol_5} \n
# max_iter_5 = {max_iter_5} \n
# p_h_grid_size = {p_h_grid_size} \n
# slow_5 = {slow_5} \n""")

# supply_demand(out, p_h_min, p_h_max, p_h_grid_size, tol_5, max_iter_5, slow_5,
#               save=True, folder=folder, file=r'\outsourcer_SD')


# #########################################

# # Compare the welfare of baseline and outsourcer, print results
# w_welfare_b = np.sum(n_grid_b * w_grid_b) + u_b * b
# f_welfare_b = np.sum(n_grid_b * (bl.y_grid - w_grid_b)) - np.sum(C(v_grid_b, gamma, k))
# t_welfare_b = np.sum(n_grid_b * bl.y_grid) + u_b * b - np.sum(C(v_grid_b, gamma, k))

# welfare_p = np.sum(n_grid_p * bl.y_grid) + u_p * b - np.sum(C(v_grid_p, gamma, k))
# per_tot_w_b = t_welfare_b / welfare_p * 100

# w_welfare_o = np.sum(n_grid_o[out.y_grid < y_hat_o] * w_grid_o[out.y_grid < y_hat_o]) + n_h_o * omega_o + u_o * b
# firm_prof_h = n_grid_o * (out.y_grid - w_grid_o)
# firm_prof_o = n_grid_o * (out.y_grid - p_h_o)
# f_welfare_o = (np.sum(firm_prof_h[out.y_grid < y_hat_o]) + np.sum(firm_prof_o[out.y_grid >= y_hat_o])  
#                - np.sum(C(v_grid_o, gamma, k)))
# o_welfare_o = n_h_o * (p_h_o - omega_o) - C_h(v_h_o, gamma_h, k_h)
# t_welfare_o = np.sum(n_grid_o * out.y_grid) - np.sum(C(v_grid_o, gamma, k)) - C_h(v_h_o, gamma_h, k_h)

# welfare_o_p = (np.sum(n_grid_o_p * out.y_grid) + u_o_p * b 
#                - np.sum(C(v_grid_o_p, gamma, k)) - C_h(v_h_o_p, gamma_h, k_h))
# per_tot_w_o = t_welfare_o / welfare_o_p * 100

# f.write("Comparing Welfare \n")
# f.write(f"""For the baseline decentralized case \n
# worker welfare is {w_welfare_b:.4f} \n
# firm welfare is {f_welfare_b:.4f} \n 
# total welfare is {t_welfare_b:.4f} \n
# For the baseline planner's problem, total welfare is {welfare_p:.4f} \n
# The decentralized solution captures {per_tot_w_b:2.0f} percent of efficient welfare \n
# For the outsourcer's decentralized case \n
# worker welfare is {w_welfare_o:.4f} \n
# firm welfare is {f_welfare_o:.4f} \n 
# outsourcer welfare is {o_welfare_o:.4f} \n 
# total welfare is {t_welfare_o:.4f} \n
# For the baseline planner's problem, total welfare is {welfare_o_p:.4f} \n
# The decentralized solution captures {per_tot_w_o:2.0f} percent of efficient welfare \n
# """)


# f.close()

# #############################################
# ########################

# # Now graph figures of interest

# # Wages, Vacancies, and Matches in baseline decentralized
# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, w_grid_b, lw=2, alpha=0.7)
# ax.set(xlabel="Firm Productivity $y$", ylabel="Wage $w$")

# fig.savefig(folder + r'\baseline_wages.pdf')


# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, v_grid_b, lw=2, alpha=0.7)
# ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

# fig.savefig(folder + r'\baseline_vacancies.pdf')


# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, n_grid_b, lw=2, alpha=0.7)
# ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

# fig.savefig(folder + r'\baseline_matches.pdf')

# plt.close(fig='all')

# # Vacancies and matches in baseline decentralized vs planner's

# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, v_grid_b, lw=2, alpha=0.7, label='Decentralized')
# ax.plot(bl.y_grid, v_grid_p, lw=2, alpha=0.7, label="Planner's")
# ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_d_p_vacancies.pdf')



# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, n_grid_b, lw=2, alpha=0.7, label='Decentralized')
# ax.plot(bl.y_grid, n_grid_p, lw=2, alpha=0.7, label="Planner's")
# ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_d_p_matches.pdf')

# plt.close(fig='all')

# # Wages, vacancies, and matches for outsourcers decentralized

# fig, ax = plt.subplots()
# omega_grid_o = omega_o * np.ones(y_grid_size)
# p_h_grid_o = p_h_o * np.ones(y_grid_size)
# fig, ax = plt.subplots(figsize=(6,6))
# ax.plot(out.y_grid[out.y_grid < y_hat_o], w_grid_o[out.y_grid < y_hat_o], lw=2, alpha=0.7, c='b')
# ax.plot(out.y_grid[out.y_grid >= y_hat_o], w_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='b', linestyle=':')
# ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':', alpha=0.7)
# ax.plot(out.y_grid[out.y_grid >= y_hat_o], omega_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='b')
# ax.plot(out.y_grid[out.y_grid >= y_hat_o], p_h_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='k')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Wages $w$")

# fig.savefig(folder + r'\outsourcer_wages.pdf')


# fig, ax = plt.subplots()
# ax.plot(out.y_grid, v_grid_o, lw=2, alpha=0.7)
# ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

# fig.savefig(folder + r'\outsourcer_vacancies.pdf')


# fig, ax = plt.subplots()
# ax.plot(out.y_grid, n_grid_o, lw=2, alpha=0.7)
# ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))

# fig.savefig(folder + r'\outsourcer_matches.pdf')

# plt.close(fig='all')

# # Vacancies and matches for outsourcers decentralized vs planner's
# fig, ax = plt.subplots()
# ax.plot(out.y_grid, v_grid_o, lw=2, alpha=0.7, label="Decentralized", color='blue')
# ax.plot(out.y_grid, v_grid_o_p, lw=2, alpha=0.7, label="Planner's", color='orange')
# ax.axvline(x=y_hat_o, c='green', lw=1, linestyle=':')
# ax.axvline(x=y_hat_o_p, c='red', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\outsourcer_d_p_vacancies.pdf')


# fig, ax = plt.subplots()
# ax.plot(out.y_grid, n_grid_o, lw=2, alpha=0.7, label="Decentralized", color='blue')
# ax.plot(out.y_grid, n_grid_o_p, lw=2, alpha=0.7, label="Planner's", color='orange')
# ax.axvline(x=y_hat_o, c='green', lw=1, linestyle=':')
# ax.axvline(x=y_hat_o_p, c='red', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\outsourcer_d_p_matches.pdf')

# plt.close(fig='all')

# # Wages, vacancies, and matches for baseline vs outsourcers decentralized

# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, w_grid_b, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid[out.y_grid < y_hat_o], w_grid_o[out.y_grid < y_hat_o], lw=2, alpha=0.7, c='orange', label='Outsourcers')
# ax.plot(out.y_grid[out.y_grid >= y_hat_o], w_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='orange', linestyle=':')
# ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':', alpha=0.7)
# ax.plot(out.y_grid[out.y_grid >= y_hat_o], omega_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='orange')
# ax.plot(out.y_grid[out.y_grid >= y_hat_o], p_h_grid_o[out.y_grid >= y_hat_o], lw=2, alpha=0.7, c='k')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Wages $w$")
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_v_outsourcer_wages.pdf')


# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, v_grid_b, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid, v_grid_o, lw=2, alpha=0.7, label="Outsourcers")
# ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_v_outsourcer_vacancies.pdf')


# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, n_grid_b, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid, n_grid_o, lw=2, alpha=0.7, label="Outsourcers")
# ax.axvline(x=y_hat_o, c='k', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_v_outsourcer_matches.pdf')

# plt.close(fig='all')

# # Vacancies and matches for baseline vs outsourcers planner's

# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, v_grid_p, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid, v_grid_o_p, lw=2, alpha=0.7, label="Outsourcers")
# ax.axvline(x=y_hat_o_p, c='k', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Vacancies $v$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_v_outsourcer_p_vacancies.pdf')


# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, n_grid_p, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid, n_grid_o_p, lw=2, alpha=0.7, label="Outsourcers")
# ax.axvline(x=y_hat_o_p, c='k', lw=1, linestyle=':')
# ax.set(xlabel="Firm Productivity $y$", ylabel="Matches $n$")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_v_outsourcer_p_matches.pdf')

# plt.close(fig='all')

# # # Vacancies and matches decentralized percent deviation from planners for baseline vs outsourcers

# # v_grid_ratio_b = (v_grid_b - v_grid_p) / v_grid_p * 100
# # v_grid_ratio_o = (v_grid_o - v_grid_o_p) / v_grid_o_p * 100

# # n_grid_ratio_b = (n_grid_b - n_grid_p) / n_grid_p * 100
# # n_grid_ratio_o = (n_grid_o - n_grid_o_p) / n_grid_o_p * 100

# # fig, ax = plt.subplots()
# # ax.plot(bl.y_grid, v_grid_ratio_b, lw=2, alpha=0.7, label="Baseline")
# # ax.plot(out.y_grid, v_grid_ratio_o, lw=2, alpha=0.7, label="Outsourcers")
# # ax.set(xlabel="Firm Productivity $y$", ylabel="Percent Deviation")
# # ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# # ax.legend(loc='best')

# # fig.savefig(folder + r'\baseline_outsourcer_d_p_ratio_vacancies.pdf')


# # fig, ax = plt.subplots()
# # ax.plot(bl.y_grid, n_grid_ratio_b, lw=2, alpha=0.7, label="Baseline")
# # ax.plot(out.y_grid, n_grid_ratio_o, lw=2, alpha=0.7, label="Outsourcers")
# # ax.set(xlabel="Firm Productivity $y$", ylabel="Percent Deviation")
# # ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# # ax.legend(loc='best')

# # fig.savefig(folder + r'\baseline_outsourcer_d_p_ratio_matches.pdf')

# # plt.close(fig='all')

# # Use differemces not ratios as goes near 0

# v_grid_dif_b = v_grid_b - v_grid_p
# v_grid_dif_o = v_grid_o - v_grid_o_p

# n_grid_dif_b = n_grid_b - n_grid_p
# n_grid_dif_o = n_grid_o - n_grid_o_p 

# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, v_grid_dif_b, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid, v_grid_dif_o, lw=2, alpha=0.7, label="Outsourcers")
# ax.set(xlabel="Firm Productivity $y$", ylabel="Difference")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_outsourcer_d_p_dif_vacancies.pdf')


# fig, ax = plt.subplots()
# ax.plot(bl.y_grid, n_grid_dif_b, lw=2, alpha=0.7, label="Baseline")
# ax.plot(out.y_grid, n_grid_dif_o, lw=2, alpha=0.7, label="Outsourcers")
# ax.set(xlabel="Firm Productivity $y$", ylabel="Difference")
# ax.ticklabel_format(axis='y', style='sci', scilimits=(1,-1))
# ax.legend(loc='best')

# fig.savefig(folder + r'\baseline_outsourcer_d_p_dif_matches.pdf')

# plt.close(fig='all')

Error at iter 1000 is 0.3496603138462625
Error at iter 2000 is 0.1285684976252499
Error at iter 3000 is 0.4175126445981064
Error at iter 4000 is 0.11467284861449734
Error at iter 5000 is 0.03306322179922544
Error at iter 6000 is 0.010343558007187531
Error at iter 7000 is 0.0027823228290497493
Error at iter 8000 is 0.0008045785488295099
Error at iter 9000 is 0.00023057345014378117
Error at iter 10000 is 6.482869071400277e-05
Error at iter 11000 is 1.770334566790488e-05
Error at iter 12000 is 4.621327363646399e-06
Error at iter 13000 is 1.117889805724026e-06
Error at iter 14000 is 2.3203263528615015e-07
Error at iter 15000 is 3.0194843084196954e-08
Converged in 15333 iterations


NameError: name 'b_spread' is not defined

In [13]:
update_grids, _, _ = p_operator_factor(bl)
f = bl.f

guess = [v_grid_b, u_b]
tol = 1e-8
max_iter = 3e3
tol_l = 1e-8
max_iter_l = 1e6
ps = 2e2
slow = 1e-1
slow_l = 1e-2 
verbose = True

# Set up initial guesses and loop parameters
#     # Use intitial guess from decentralized problem
#     v_grid, u = solve_model(bl, tol=1e-8, verbose=False, print_skip=1000, max_iter=100000, slow=1/1000)
v_grid, u = guess
theta = np.sum(v_grid) / u
n_grid = (bl.q(theta) * v_grid) / bl.delta

# Carry around lam to make re-estimating faster
lam = (bl.y_grid - bl.b) / (bl.r + bl.delta)

i = 0
err = tol + 1 

# Update v_grid, n_grid, and lamda each period
while i < max_iter and err > tol:
    v_grid_new, n_grid_new, lam_new = update_grids(v_grid, n_grid, lam, tol_l, max_iter_l, slow, slow_l)
    err_v = np.max(np.abs(v_grid_new - v_grid))
    err_n = np.max(np.abs(n_grid_new - n_grid))
    err = max(err_v, err_n)
    i += 1

    if verbose and i % print_skip == 0:
        print(f'Error at iter {i} is {err}')
        print(v_grid[-1])

    if verbose and i % print_skip == 1:
        print(f'Error at iter {i} is {err}')
        print(v_grid[-1]) 

#         if verbose and i >= 0 and i <= 5:
#             print(f'Error at iter {i} is {err}')
#             print(v_grid[-1])

    v_grid, n_grid = v_grid_new * slow + v_grid * (1 - slow), n_grid_new * slow + n_grid * (1 - slow)
    lam = lam_new * slow + lam * (1 - slow)


if i == max_iter:
    print("Failed to Converge")

if verbose and i < max_iter:
    print(f'Converged in {i} iterations')

Error at iter 1 is 0.0003699807320475756
1.1304565038660958e-05
Converged in 101 iterations


In [17]:
print(v_grid, n_grid, 1 - np.sum(n_grid))
print(C(v_grid[-1], gamma, k_h))
print(lam)
u = 1 - np.sum(n_grid)
theta = np.sum(v_grid) / u
print(c_inv(lam * bl.q(theta) + ((bl.q_p(theta) / u) * np.sum(v_grid * lam)), gamma, k))
print(lam * bl.q(theta) + ((bl.q_p(theta) / u) * np.sum(v_grid * lam)))
print(theta)
print(bl.q_p(theta))

[6.73460393e-11 6.73967751e-11 6.74475109e-11 ... 2.70137083e-10
 2.70187819e-10 2.70238555e-10] [2.20413053e-09 2.20579103e-09 2.20745154e-09 ... 8.84116422e-09
 8.84282472e-09 8.84448522e-09] 0.9999779027684962
7.302887667242386e-18
[57.65781307 57.66386554 57.66991801 ... 81.84952979 81.85558226
 81.86163473]
[0. 0. 0. ... 0. 0. 0.]
[-26297.13042002 -26297.12436755 -26297.11831508 ... -26272.9387033
 -26272.93265083 -26272.92659836]
6.751841084477531e-07
-540739302.929368
